In [134]:
# Renameios
#!/usr/bin/python
#
# Renaming iOS devices and push install Khan Academy in MDM
#
# Single threading version 0.1
#
# Usage:
#
# Author:Kelvin J. July,2017
#
#
import pip 
import csv,sys,json,os,re
import urllib 
import time

try:
    from robobrowser import RoboBrowser
except :
    pip.main(['install', 'robobrowser'])
from StringIO import StringIO
import getpass,time
#import threading

try:
    from enum import Enum
except :
    pip.main(['install', 'enum'])
try:
    import lxml
except :
    pip.main(['install', 'lxml'])

try:
    from bs4 import BeautifulSoup
except :
    pip.main(['install', 'bs4'])

    
    
class Roles(Enum):
    Elementary = '18536'
    Secondary = '18985'
    TCDSB_TOP_LEVEL= '18535'
     
class OUs(Enum):
    ios = 'ios'
    tv = 'tv'
    ioslease= 'ioslease'
    elementary='2717'
    tcdsb='2673'
    secondary='2722'


class idevice(object):
    def __init__(self):
        self.attrs={'DeviceOwnership':None,
            'UserName':None,
                'EmailAddress':None,
                'Platform':None,
                'FriendlyName':None,
                'OSVersion':None,
                'DisplayModel':None ,
                'EnrollmentStatusName':None,
                'ComplianceStatusName':None ,
                'DisplayName':None,
                'ManagedBy':None,
                'AssetNumber':None,
                'SerialNumber':None,
                'LocationGroupName':None,
                'IsCompromisedYN': None,
                'WiFiIPAddress':None,
                'WiFiMacAddress':None,
                'Notes':None,
                'WnsStatus' :None,
                'DeviceId':None}

    def getAttrs(self):
		return self.attrs

def searchAll(browser,querystr):
    devices={}
    text=querystr
    text=urllib.quote_plus(text)
    
    queryurl='https://cn239.awmdm.com/AirWatch/Device/List/Search?Page=&Size=100&SearchText='+text+'ipads+ipad&CurrentView=Custom&Order=Descending&Sort=LastPingDate'
    #queryurl='https://cn239.awmdm.com/AirWatch/Device/List/Search?Page=&Size=10&SearchText='+text+'&CurrentView=Custom&Order=Descending&Sort=LastPingDate'

 
    #print queryurl
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Referer']='https://cn239.awmdm.com/AirWatch/'

  
    browser.open(queryurl)
 
    
    try:          
        result_trs= browser.find("div",{"id": "DeviceGrid"}).findAll('tr',{'data-action-names':"QueryAll LockDevice SendMessage ChangeOrganizationGroup DeleteDevice"})
        if len(result_trs)>0:
            print "Number of devices found: "+str(len(result_trs)) 
        else:
            print "Number of devices found: 0"
        for r in result_trs: 
            dev_id=None
            dev_sn=None
            tr=BeautifulSoup(str(r),'lxml')
            tags=tr.findAll('td')
            for t in tags:      
                if t.has_attr('data-property') and t.attrs['data-property']== 'SerialNumber':
                    dev_sn=t.text

            tags=tr.findAll('input')
            for t in tags:
                if t.attrs['type']=='checkbox':
                    #print t.attrs['value']
                    dev_id=t.attrs['value']
            devices[dev_sn]=dev_id 
    except:
        print 'Error on finding serial number...'
        #sys.exit(1)
    
    #src = str(browser.parsed())

    #result = re.search('%s(.*)%s' % (start, end), src).group(1)
    #print devices  
    with open('sns.json', 'w') as file:
         file.write(json.dumps(devices))
    return devices

def queryOne(browser,querystr):
    
    text=querystr.strip()
    text=urllib.quote_plus(text)
    
    queryurl='https://cn239.awmdm.com/AirWatch/Device/List/Search?Page=&Size=100&SearchText='+text
    
    #print queryurl
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Referer']='https://cn239.awmdm.com/AirWatch/'
    
    browser.open(queryurl)
    #print browser.response.content
    
    #     try:
    
    result_trs= browser.find("div",{"id": "DeviceGrid"}).findAll('tr',{'data-action-names':"QueryAll LockDevice SendMessage ChangeOrganizationGroup DeleteDevice"})
    if len(result_trs)>0:
        print "Number of devices found: "+str(len(result_trs))
    else:
        sys.exit('Number of devices found: 0')
    
    for r in result_trs:
        device=idevice()
        
        tr=BeautifulSoup(str(r),'lxml')
        tags=tr.findAll('td')
        for t in tags:     #if t.has_attr('data-property') and t.attrs['data-property']== 'SerialNumber':
            if t.has_attr('data-property'):
                if t.attrs['data-property'] in device.getAttrs():
                    if t.attrs['data-property']=='FriendlyName':
                        device.attrs['FriendlyName']=(t.find('a').text)
                    else:
                        device.attrs[t.attrs['data-property']]=t.text
        
        tags=tr.findAll('input')
        for t in tags:
            if t.attrs['type']=='checkbox':
                #print t.attrs['value']
                device.attrs['DeviceId']=t.attrs['value']

#     except:
#         print 'Error on retriving device info...'
#         sys.exit(1)

    if device.attrs['SerialNumber'] !=querystr:
        sys.exit('Returned device s/n does not match the one that was searched...')
    
    return device

def createBrowser(user,passwd):
    
    
    browser = RoboBrowser(history=True)
    
    browser.open('https://cn239.awmdm.com/AirWatch/Login/Login/Login-User')
    
    loginForm = browser.get_form()
    loginForm['UserName']=user
    loginForm['Password']=passwd
    
    browser.submit_form(loginForm)
    resp = json.load(StringIO(browser.response.content))
    
    _success=resp['IsSuccess']
    
    if _success == False:
        print "Login failed...."
        sys.exit(1)
    
    return browser

#def rename(browser,schoolcode,device):
def rename(browser,schoolcode,device_sn,device_id):    
    newname='A'+str(schoolcode)+device_sn
    
    #browser.open('https://cn239.awmdm.com/AirWatch')
    #setOU('18536',findToken(browser),browser)
    
    url='https://cn239.awmdm.com/AirWatch/Devices/DeviceDetails.EditDevice/'+str(device_id)
     
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    #browser.session.headers['hash']='#/Devices/DeviceDetails.EditDevice/450182'
    browser.session.headers['Referer']='https://cn239.awmdm.com/AirWatch'
    
    browser.open(url)
    
    #response=browser.response.content
    #print response
    form=browser.get_form()
    
    form['FriendlyName']=newname
    browser.submit_form(form)
    resp = json.load(StringIO(browser.response.content))
    
    #print resp
    _success=resp['IsSuccess']
    
    if _success == False:
        print "Changing name failed...."
    else:
        print "Changing name is successful..."

def findToken(browser):
    
    data  = browser.find_all("script")
    paragraphs = []
    
    for x in data:
        paragraphs.append(str(x)) 
     
    regex='(?<=value=&quot;).{151}'
    matches = re.finditer(regex, " ".join(str(x) for x in paragraphs)) 
    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
        #print match.group()
        return match.group()
    
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1

#             print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))


def installKhan(browser,device_id,appid):
    app_id=appid
    app_url='https://cn239.awmdm.com/AirWatch/#/AirWatch/Device/Details/Apps/'+str(device_id)
    #app_url_install='https://cn239.awmdm.com/AirWatch/Devices/InstallApp?deviceId='+str(device_id)+'&deviceTypeId=2&selectedPrimaryKeys=7053|org.khanacademy.Khan-Academy|False'
    app_url_install='https://cn239.awmdm.com/AirWatch/Devices/InstallApp'
    app_url_confirm='https://cn239.awmdm.com/AirWatch/Devices/InstallApp?deviceId='+str(device_id)+'&appId='+app_id+'&deviceTypeId=2&source=2'
 
    browser.open(app_url)
    
    token=findToken(browser)
    
    data={'deviceId':device_id,'__RequestVerificationToken':token,'deviceTypeId':'2','selectedPrimaryKeys':app_id+'|org.khanacademy.Khan-Academy|False'}
          
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    #browser.session.headers['hash']='#/Devices/DeviceDetails.EditDevice/450182'
    browser.session.headers['Referer']='https://cn239.awmdm.com/AirWatch'
     
    #browser.session.post(app_url_install,token)
    browser.session.post(app_url_install,data)
     
    browser.open(app_url_confirm)
    
    #print browser.response.content
    
    try:
        resp = json.load(StringIO(browser.response.content))
    except:
        print device_id 
        print browser.response.content
        
    _success=resp['IsSuccess']

    if _success == True:
        print "Installing Khan is successful...." 
    else: 
        print "Installing Khan failed...." 
        
def requestOsUpdate(browser,device_id):
    update_url='https://cn239.awmdm.com/AirWatch/Devices/RequestOsUpdates/'
    token=findToken(browser)
    browser.session.post(update_url+device_id,token)
    #print browser.response.content
    
def setRole(role_id,browser):
    print 'setting OU to '+role_id
    RoleCookies={
    'ic4DUh/NXVp8VOKAtyDgbA==':'Elementary_Top_Level',
    'MVHG2D97GH8n5jFRanbzhQ==':'TCDSB_Top_Level',
    '0od0Xt3gw7UG3qUZKSaZvA==':'Secondary_Top_Level'
    }
    
    browser.open('https://cn239.awmdm.com/AirWatch/')
    token=findToken(browser)
    
    url='https://cn239.awmdm.com/AirWatch/Users/set-role'
    data={'id':role_id,'__RequestVerificationToken':token}
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Accept']='*/*' 
    browser.session.headers['Host']='cn239.awmdm.com'
    browser.session.headers['Origin']='https://cn239.awmdm.com'
    browser.session.headers['Pragma']='no-cache'
    browser.session.headers['Referer']='https://cn239.awmdm.com/AirWatch/'
    browser.session.headers['Connection']='keep-alive' 
    browser.session.headers['Content-Type']='application/x-www-form-urlencoded; charset=UTF-8'
                            
    #cookies_old=browser.session.cookies.get_dict()
    oldRole=str(browser.session.cookies['BrandingUserLocationGroupID']).strip()
    
    if oldRole in RoleCookies.keys():
        print 'Old Role: '+ str(RoleCookies[oldRole])
     
    
    #browser.open('https://cn239.awmdm.com/AirWatch/Users/set-role?id='+role_id+'&__RequestVerificationToken='+token,method='post')
    browser.session.post('https://cn239.awmdm.com/AirWatch/Users/set-role',data)
    
    newRole=str(browser.session.cookies['BrandingUserLocationGroupID']).strip()
    
    if newRole in RoleCookies.keys():
        print 'Set new role to :'+str(RoleCookies[newRole])
          
            

def main():
    schoolcode=raw_input("school code:")
    sns=[]
    appid='9423'
    #appid='71318'
    
    schoolcode=schoolcode.strip()
    # user = raw_input("Username:")
    # passwd = getpass.getpass("Password for " + user + ":")
    user='tcdsb.org\jiangk'
    passwd='2793587j'
    
    browser=createBrowser(user,passwd)
    
    #setRole(Roles.Elementary.value,browser)
    
    count=0
    err_devices=[]
     
    devices = searchAll(browser,schoolcode) 
    
    #print devices
    global deviceIds
    deviceIds=[]
    count=0
    for k in devices:
        
        try:
            count+=1
            print 'Renaming number '+str(count)+' of '+str(len(devices))
            #device=queryOne(browser,k.strip())
            time.sleep(2)
            #print 'Renaming '+device.attrs['FriendlyName'] +' to A'+str(schoolcode)+device.attrs['SerialNumber']
            #rename(browser,schoolcode,device)
            rename(browser,schoolcode,k,devices[k])
            deviceIds.append(devices[k])
            
            #time.sleep(2)
            
#             requestOsUpdate(browser,k) 
#             time.sleep(5)
            #installKhan(browser,devices[k])
             
        except:
            err_device.append(k)
            
    #time.sleep(60)
#     for k in deviceIds:
#         installKhan(browser,k,appid)
        
#     time.sleep(10)
    
#     for k in deviceIds:
#         installKhan(browser,k,appid)
    
    print 'Proccessed '+str(count)+' devices'
    if len(err_devices)>0:
        print 'Devices with errors:'
        for d in err_devices:
            print d
            
    print  "Finished....."

if __name__ == '__main__':
    main()



school code:0556
Number of devices found: 13
Renaming number 1 of 13
Changing name is successful...
Renaming number 2 of 13
Changing name is successful...
Renaming number 3 of 13
Changing name is successful...
Renaming number 4 of 13
Changing name is successful...
Renaming number 5 of 13
Changing name is successful...
Renaming number 6 of 13
Changing name is successful...
Renaming number 7 of 13
Changing name is successful...
Renaming number 8 of 13
Changing name is successful...
Renaming number 9 of 13
Changing name is successful...
Renaming number 10 of 13
Changing name is successful...
Renaming number 11 of 13
Changing name is successful...
Renaming number 12 of 13
Changing name is successful...
Renaming number 13 of 13
Changing name is successful...
Proccessed 13 devices
Finished.....


In [43]:
devices

NameError: name 'devices' is not defined

In [28]:
###
# pushing twice khan 
###


    
    
def installKhan(browser,device_id,appid):
    app_id=appid
    app_url='https://cn239.awmdm.com/AirWatch/#/AirWatch/Device/Details/Apps/'+str(device_id)
    #app_url_install='https://cn239.awmdm.com/AirWatch/Devices/InstallApp?deviceId='+str(device_id)+'&deviceTypeId=2&selectedPrimaryKeys=7053|org.khanacademy.Khan-Academy|False'
    app_url_install='https://cn239.awmdm.com/AirWatch/Devices/InstallApp'
    app_url_confirm='https://cn239.awmdm.com/AirWatch/Devices/InstallApp?deviceId='+str(device_id)+'&appId='+app_id+'&deviceTypeId=2&source=2'
 
    browser.open(app_url)
    
    token=findToken(browser)
    
    data={'deviceId':device_id,'__RequestVerificationToken':token,'deviceTypeId':'2','selectedPrimaryKeys':app_id+'|org.khanacademy.Khan-Academy|False'}
          
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    #browser.session.headers['hash']='#/Devices/DeviceDetails.EditDevice/450182'
    browser.session.headers['Referer']='https://cn239.awmdm.com/AirWatch'
     
    #browser.session.post(app_url_install,token)
    browser.session.post(app_url_install,data)
     
    browser.open(app_url_confirm)
    
    #print browser.response.content
    
    try:
        resp = json.load(StringIO(browser.response.content))
    except:
        print device_id 
        print browser.response.content
        
    _success=resp['IsSuccess']

    if _success == True:
        print "Installing Khan is successful...." 
    else: 
        print "Installing Khan failed...." 
        
user='tcdsb.org\jiangk'
passwd='2793587j'

browser=createBrowser(user,passwd)
appid='10355'
#setRole(Roles.Elementary.value,browser) 
for k in deviceIds:
    installKhan(browser,k,appid)
        
time.sleep(10)

for k in deviceIds:
    installKhan(browser,k,appid)    
    
print 'Finished...'

Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Installing Khan is successful....
Finished...


In [6]:
# Move OU and move back

import pip 
import csv,sys,json,os,re
import urllib 
import time

try:
    from robobrowser import RoboBrowser
except :
    pip.main(['install', 'robobrowser'])
from StringIO import StringIO
import getpass,time
#import threading

try:
    from enum import Enum
except :
    pip.main(['install', 'enum'])
try:
    import lxml
except :
    pip.main(['install', 'lxml'])

try:
    from bs4 import BeautifulSoup
except :
    pip.main(['install', 'bs4'])
    
    
    
class Roles(Enum):
    Elementary = '18536'
    Secondary = '18985'
    TCDSB_TOP_LEVEL= '18535'
     
class OUs(Enum):
    ios = 'ios'
    tv = 'tv'
    ioslease= 'ioslease'
    elementary='2717'
    tcdsb='2673'
    secondary='2722'
    
def chunks(data, SIZE=10):
    it = iter(data)
    for i in xrange(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
        
######################################################################
# Modified search method to query by school code for leased iPads
######################################################################

def searchAll(browser,schoolcode):
    devices={}
    text=schoolcode
    text=urllib.quote_plus(text)
    
    queryurl='https://cn239.awmdm.com/AirWatch/Device/List/Search?Page=&Size=10&SearchText='+text+'&CurrentView=Custom&Order=Descending&Sort=LastPingDate'
 
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Referer']='https://cn239.awmdm.com/AirWatch/'

   
    browser.open(queryurl)
      
    #print browser.response.content
     
    pattern = re.compile("^[A,a]"+schoolcode.strip()+"[a-zA-Z0-9]{12}$") 

    try:
                 
        result_trs= browser.find("div",{"id": "DeviceGrid"}).findAll('tr',{'data-action-names':"QueryAll LockDevice SendMessage ChangeOrganizationGroup DeleteDevice"})
          
        if len(result_trs)>0:
            print "Number of devices found: "+str(len(result_trs)) 
        else:
            print "Number of devices found: 0"
            sys.exit('Exiting....')
            
        for r in result_trs: 
            dev_id=None
            dev_sn=None
            tr=BeautifulSoup(str(r),'lxml')
            tags_td=tr.findAll('td')
            
            tags_input=tr.findAll('input')
            for t in tags_input:
                if t.attrs['type']=='checkbox':
                    #print t.attrs['value']
                    dev_id=t.attrs['value']
            ###
            ### find device that matches new naming convention
            ###
            
            for t in tags_td:    
                if t.has_attr('data-property') and t.attrs['data-property']== 'FriendlyName':
                    if len(t.text.strip().split())<2:
                        if pattern.match(t.text.split()[0]) != None: 
                            print t.text.split()[0]

                            dev_sn=t.text.split()[0].strip()[5:]
                            #print dev_sn
                            #sys.exit(1)
#                         if t.has_attr('data-property') and t.attrs['data-property']== 'SerialNumber':
#                             print t.text.strip()
                        else: continue     
            if dev_sn !=None:
                devices[dev_sn]=dev_id 
    except:
        print 'Error on finding serial number...'
        sys.exit()
    
    #src = str(browser.parsed())

    #result = re.search('%s(.*)%s' % (start, end), src).group(1)
    #print devices  
#     if os.path.exists('sns.json'):
#         os.remove('sns.json')
#     with open('sns.json', 'w') as file:
#          file.write(json.dumps(devices))
    if len(devices)==0:
         sys.exit('Didn\'t find any devices matching naming convention')
    return devices

       
# MoveOU and move back
def moveOU(browser,devices,schoolcode,group_type):
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
 
    if group_type==None or schoolcode==None:
        sys.exit('Error on moving to group: school code or device_type')
    lst=[]
    for g in chunks(devices):
        lst.append(g)
     
    grp='[{"LocationGroupId":2673,"LocationGroupName":"Toronto Catholic District School Board","ParentId":576,"label":"Toronto Catholic District School Board"},{"LocationGroupId":4568,"LocationGroupName":"_Development","ParentId":2673,"label":"Toronto Catholic District School Board / _Development"},{"LocationGroupId":17600,"LocationGroupName":"AppleTV","ParentId":4568,"label":"Toronto Catholic District School Board / _Development / AppleTV"},{"LocationGroupId":10553,"LocationGroupName":"Dev_Telecom","ParentId":4568,"label":"Toronto Catholic District School Board / _Development / Dev_Telecom"},{"LocationGroupId":17955,"LocationGroupName":"Playground","ParentId":4568,"label":"Toronto Catholic District School Board / _Development / Playground"},{"LocationGroupId":3328,"LocationGroupName":"_Others","ParentId":2673,"label":"Toronto Catholic District School Board / _Others"},{"LocationGroupId":3306,"LocationGroupName":"BYOD","ParentId":2673,"label":"Toronto Catholic District School Board / BYOD"},{"LocationGroupId":3278,"LocationGroupName":"CEC","ParentId":2673,"label":"Toronto Catholic District School Board / CEC"},{"LocationGroupId":7064,"LocationGroupName":"AdultEducationPrograms_0783 ","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / AdultEducationPrograms_0783 "},{"LocationGroupId":7065,"LocationGroupName":"ParkPlaceLinc_6593","ParentId":7064,"label":"Toronto Catholic District School Board / CEC / AdultEducationPrograms_0783  / ParkPlaceLinc_6593"},{"LocationGroupId":7068,"LocationGroupName":"iOS","ParentId":7065,"label":"Toronto Catholic District School Board / CEC / AdultEducationPrograms_0783  / ParkPlaceLinc_6593 / iOS"},{"LocationGroupId":13641,"LocationGroupName":"Autism_1327","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / Autism_1327"},{"LocationGroupId":13642,"LocationGroupName":"iOS","ParentId":13641,"label":"Toronto Catholic District School Board / CEC / Autism_1327 / iOS"},{"LocationGroupId":13643,"LocationGroupName":"CommunityRelation_0808","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / CommunityRelation_0808"},{"LocationGroupId":13644,"LocationGroupName":"iOS","ParentId":13643,"label":"Toronto Catholic District School Board / CEC / CommunityRelation_0808 / iOS"},{"LocationGroupId":7175,"LocationGroupName":"Curriculum\u0026Accountability_0737","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / Curriculum\u0026Accountability_0737"},{"LocationGroupId":7176,"LocationGroupName":"iOS","ParentId":7175,"label":"Toronto Catholic District School Board / CEC / Curriculum\u0026Accountability_0737 / iOS"},{"LocationGroupId":13645,"LocationGroupName":"EarlyLearningProgram_1377","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / EarlyLearningProgram_1377"},{"LocationGroupId":13646,"LocationGroupName":"iOS","ParentId":13645,"label":"Toronto Catholic District School Board / CEC / EarlyLearningProgram_1377 / iOS"},{"LocationGroupId":13650,"LocationGroupName":"EmployeeRelations_HealthSafety_0774","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / EmployeeRelations_HealthSafety_0774"},{"LocationGroupId":13651,"LocationGroupName":"iOS","ParentId":13650,"label":"Toronto Catholic District School Board / CEC / EmployeeRelations_HealthSafety_0774 / iOS"},{"LocationGroupId":7303,"LocationGroupName":"FacilitiesE_0849","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / FacilitiesE_0849"},{"LocationGroupId":7305,"LocationGroupName":"Android","ParentId":7303,"label":"Toronto Catholic District School Board / CEC / FacilitiesE_0849 / Android"},{"LocationGroupId":7304,"LocationGroupName":"iOS","ParentId":7303,"label":"Toronto Catholic District School Board / CEC / FacilitiesE_0849 / iOS"},{"LocationGroupId":13652,"LocationGroupName":"FrenchDept_0711","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / FrenchDept_0711"},{"LocationGroupId":13653,"LocationGroupName":"iOS","ParentId":13652,"label":"Toronto Catholic District School Board / CEC / FrenchDept_0711 / iOS"},{"LocationGroupId":7042,"LocationGroupName":"LanguageArtsDept_0706","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / LanguageArtsDept_0706"},{"LocationGroupId":7069,"LocationGroupName":"iOS","ParentId":7042,"label":"Toronto Catholic District School Board / CEC / LanguageArtsDept_0706 / iOS"},{"LocationGroupId":6961,"LocationGroupName":"MathematicsDept_0704","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / MathematicsDept_0704"},{"LocationGroupId":7070,"LocationGroupName":"iOS","ParentId":6961,"label":"Toronto Catholic District School Board / CEC / MathematicsDept_0704 / iOS"},{"LocationGroupId":16766,"LocationGroupName":"Religion_0700","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / Religion_0700"},{"LocationGroupId":16768,"LocationGroupName":"iOS","ParentId":16766,"label":"Toronto Catholic District School Board / CEC / Religion_0700 / iOS"},{"LocationGroupId":7047,"LocationGroupName":"ResearchDept_0753","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / ResearchDept_0753"},{"LocationGroupId":7071,"LocationGroupName":"iOS","ParentId":7047,"label":"Toronto Catholic District School Board / CEC / ResearchDept_0753 / iOS"},{"LocationGroupId":8827,"LocationGroupName":"SafeSchools_0764","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / SafeSchools_0764"},{"LocationGroupId":8828,"LocationGroupName":"iOS","ParentId":8827,"label":"Toronto Catholic District School Board / CEC / SafeSchools_0764 / iOS"},{"LocationGroupId":13654,"LocationGroupName":"SchoolEffectivenessLead_1252","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / SchoolEffectivenessLead_1252"},{"LocationGroupId":13655,"LocationGroupName":"iOS","ParentId":13654,"label":"Toronto Catholic District School Board / CEC / SchoolEffectivenessLead_1252 / iOS"},{"LocationGroupId":12785,"LocationGroupName":"SEA_0771","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / SEA_0771"},{"LocationGroupId":12786,"LocationGroupName":"iOS","ParentId":12785,"label":"Toronto Catholic District School Board / CEC / SEA_0771 / iOS"},{"LocationGroupId":17895,"LocationGroupName":"Mac","ParentId":12785,"label":"Toronto Catholic District School Board / CEC / SEA_0771 / Mac"},{"LocationGroupId":10365,"LocationGroupName":"SpecialEd_Sec23_0769","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / SpecialEd_Sec23_0769"},{"LocationGroupId":10371,"LocationGroupName":"iOS","ParentId":10365,"label":"Toronto Catholic District School Board / CEC / SpecialEd_Sec23_0769 / iOS"},{"LocationGroupId":12320,"LocationGroupName":"SpecialServices_0717","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / SpecialServices_0717"},{"LocationGroupId":12321,"LocationGroupName":"iOS","ParentId":12320,"label":"Toronto Catholic District School Board / CEC / SpecialServices_0717 / iOS"},{"LocationGroupId":11881,"LocationGroupName":"Speech\u0026Language_0777","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / Speech\u0026Language_0777"},{"LocationGroupId":11882,"LocationGroupName":"iOS","ParentId":11881,"label":"Toronto Catholic District School Board / CEC / Speech\u0026Language_0777 / iOS"},{"LocationGroupId":13656,"LocationGroupName":"StudentSuccess_1102","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / StudentSuccess_1102"},{"LocationGroupId":13657,"LocationGroupName":"iOS","ParentId":13656,"label":"Toronto Catholic District School Board / CEC / StudentSuccess_1102 / iOS"},{"LocationGroupId":7164,"LocationGroupName":"Superintendents_8282","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / Superintendents_8282"},{"LocationGroupId":7180,"LocationGroupName":"iOS","ParentId":7164,"label":"Toronto Catholic District School Board / CEC / Superintendents_8282 / iOS"},{"LocationGroupId":7994,"LocationGroupName":"SysIniStaffDevl_0762","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / SysIniStaffDevl_0762"},{"LocationGroupId":7996,"LocationGroupName":"iOS","ParentId":7994,"label":"Toronto Catholic District School Board / CEC / SysIniStaffDevl_0762 / iOS"},{"LocationGroupId":3279,"LocationGroupName":"TechnicalServices_0831","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / TechnicalServices_0831"},{"LocationGroupId":7072,"LocationGroupName":"iOS","ParentId":3279,"label":"Toronto Catholic District School Board / CEC / TechnicalServices_0831 / iOS"},{"LocationGroupId":13658,"LocationGroupName":"Trillium_0811","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / Trillium_0811"},{"LocationGroupId":13659,"LocationGroupName":"iOS","ParentId":13658,"label":"Toronto Catholic District School Board / CEC / Trillium_0811 / iOS"},{"LocationGroupId":7043,"LocationGroupName":"TrusteeServices_0842","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / TrusteeServices_0842"},{"LocationGroupId":7074,"LocationGroupName":"iOS","ParentId":7043,"label":"Toronto Catholic District School Board / CEC / TrusteeServices_0842 / iOS"},{"LocationGroupId":10184,"LocationGroupName":"iOS_Schools","ParentId":7043,"label":"Toronto Catholic District School Board / CEC / TrusteeServices_0842 / iOS_Schools"},{"LocationGroupId":16765,"LocationGroupName":"VisualArts_0714","ParentId":3278,"label":"Toronto Catholic District School Board / CEC / VisualArts_0714"},{"LocationGroupId":16767,"LocationGroupName":"iOS","ParentId":16765,"label":"Toronto Catholic District School Board / CEC / VisualArts_0714 / iOS"},{"LocationGroupId":2717,"LocationGroupName":"Elementary","ParentId":2673,"label":"Toronto Catholic District School Board / Elementary"},{"LocationGroupId":5035,"LocationGroupName":"ADevSchool_0111","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / ADevSchool_0111"},{"LocationGroupId":6831,"LocationGroupName":"0111_Test","ParentId":5035,"label":"Toronto Catholic District School Board / Elementary / ADevSchool_0111 / 0111_Test"},{"LocationGroupId":17656,"LocationGroupName":"AppleTV","ParentId":5035,"label":"Toronto Catholic District School Board / Elementary / ADevSchool_0111 / AppleTV"},{"LocationGroupId":5036,"LocationGroupName":"iOS","ParentId":5035,"label":"Toronto Catholic District School Board / Elementary / ADevSchool_0111 / iOS"},{"LocationGroupId":17927,"LocationGroupName":"iOSLease","ParentId":5035,"label":"Toronto Catholic District School Board / Elementary / ADevSchool_0111 / iOSLease"},{"LocationGroupId":3974,"LocationGroupName":"AllSaints_0290","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / AllSaints_0290"},{"LocationGroupId":18085,"LocationGroupName":"AppleTV","ParentId":3974,"label":"Toronto Catholic District School Board / Elementary / AllSaints_0290 / AppleTV"},{"LocationGroupId":4085,"LocationGroupName":"iOS","ParentId":3974,"label":"Toronto Catholic District School Board / Elementary / AllSaints_0290 / iOS"},{"LocationGroupId":17929,"LocationGroupName":"iOSLease","ParentId":3974,"label":"Toronto Catholic District School Board / Elementary / AllSaints_0290 / iOSLease"},{"LocationGroupId":3984,"LocationGroupName":"Annunciation_0299","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / Annunciation_0299"},{"LocationGroupId":18086,"LocationGroupName":"AppleTV","ParentId":3984,"label":"Toronto Catholic District School Board / Elementary / Annunciation_0299 / AppleTV"},{"LocationGroupId":4087,"LocationGroupName":"iOS","ParentId":3984,"label":"Toronto Catholic District School Board / Elementary / Annunciation_0299 / iOS"},{"LocationGroupId":17958,"LocationGroupName":"iOSLease","ParentId":3984,"label":"Toronto Catholic District School Board / Elementary / Annunciation_0299 / iOSLease"},{"LocationGroupId":6513,"LocationGroupName":"ASchool_0000","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / ASchool_0000"},{"LocationGroupId":16284,"LocationGroupName":"Admin","ParentId":6513,"label":"Toronto Catholic District School Board / Elementary / ASchool_0000 / Admin"},{"LocationGroupId":16289,"LocationGroupName":"TelecomData","ParentId":16284,"label":"Toronto Catholic District School Board / Elementary / ASchool_0000 / Admin / TelecomData"},{"LocationGroupId":16287,"LocationGroupName":"TelecomVoice","ParentId":16284,"label":"Toronto Catholic District School Board / Elementary / ASchool_0000 / Admin / TelecomVoice"},{"LocationGroupId":16290,"LocationGroupName":"TelecomVoiceData","ParentId":16284,"label":"Toronto Catholic District School Board / Elementary / ASchool_0000 / Admin / TelecomVoiceData"},{"LocationGroupId":16282,"LocationGroupName":"iOS_Purchased","ParentId":6513,"label":"Toronto Catholic District School Board / Elementary / ASchool_0000 / iOS_Purchased"},{"LocationGroupId":4029,"LocationGroupName":"BlessedJohnXXIII_0348","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / BlessedJohnXXIII_0348"},{"LocationGroupId":18087,"LocationGroupName":"AppleTV","ParentId":4029,"label":"Toronto Catholic District School Board / Elementary / BlessedJohnXXIII_0348 / AppleTV"},{"LocationGroupId":4125,"LocationGroupName":"iOS","ParentId":4029,"label":"Toronto Catholic District School Board / Elementary / BlessedJohnXXIII_0348 / iOS"},{"LocationGroupId":17959,"LocationGroupName":"iOSLease","ParentId":4029,"label":"Toronto Catholic District School Board / Elementary / BlessedJohnXXIII_0348 / iOSLease"},{"LocationGroupId":4198,"LocationGroupName":"BlessedMargherita_0409","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / BlessedMargherita_0409"},{"LocationGroupId":18088,"LocationGroupName":"AppleTV","ParentId":4198,"label":"Toronto Catholic District School Board / Elementary / BlessedMargherita_0409 / AppleTV"},{"LocationGroupId":4221,"LocationGroupName":"iOS","ParentId":4198,"label":"Toronto Catholic District School Board / Elementary / BlessedMargherita_0409 / iOS"},{"LocationGroupId":17961,"LocationGroupName":"iOSLease","ParentId":4198,"label":"Toronto Catholic District School Board / Elementary / BlessedMargherita_0409 / iOSLease"},{"LocationGroupId":7200,"LocationGroupName":"BlessedPierGiorgioFrassati_0456","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / BlessedPierGiorgioFrassati_0456"},{"LocationGroupId":18089,"LocationGroupName":"AppleTV","ParentId":7200,"label":"Toronto Catholic District School Board / Elementary / BlessedPierGiorgioFrassati_0456 / AppleTV"},{"LocationGroupId":7201,"LocationGroupName":"iOS","ParentId":7200,"label":"Toronto Catholic District School Board / Elementary / BlessedPierGiorgioFrassati_0456 / iOS"},{"LocationGroupId":17962,"LocationGroupName":"iOSLease","ParentId":7200,"label":"Toronto Catholic District School Board / Elementary / BlessedPierGiorgioFrassati_0456 / iOSLease"},{"LocationGroupId":3913,"LocationGroupName":"BlessedSacrament_0226","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / BlessedSacrament_0226"},{"LocationGroupId":18090,"LocationGroupName":"AppleTV","ParentId":3913,"label":"Toronto Catholic District School Board / Elementary / BlessedSacrament_0226 / AppleTV"},{"LocationGroupId":4126,"LocationGroupName":"iOS","ParentId":3913,"label":"Toronto Catholic District School Board / Elementary / BlessedSacrament_0226 / iOS"},{"LocationGroupId":18020,"LocationGroupName":"iOSLease","ParentId":3913,"label":"Toronto Catholic District School Board / Elementary / BlessedSacrament_0226 / iOSLease"},{"LocationGroupId":3973,"LocationGroupName":"BlessedTrinity_0289","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / BlessedTrinity_0289"},{"LocationGroupId":18091,"LocationGroupName":"AppleTV","ParentId":3973,"label":"Toronto Catholic District School Board / Elementary / BlessedTrinity_0289 / AppleTV"},{"LocationGroupId":4127,"LocationGroupName":"iOS","ParentId":3973,"label":"Toronto Catholic District School Board / Elementary / BlessedTrinity_0289 / iOS"},{"LocationGroupId":18021,"LocationGroupName":"iOSLease","ParentId":3973,"label":"Toronto Catholic District School Board / Elementary / BlessedTrinity_0289 / iOSLease"},{"LocationGroupId":3931,"LocationGroupName":"CanadianMartyrs_0251","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / CanadianMartyrs_0251"},{"LocationGroupId":18092,"LocationGroupName":"AppleTV","ParentId":3931,"label":"Toronto Catholic District School Board / Elementary / CanadianMartyrs_0251 / AppleTV"},{"LocationGroupId":4128,"LocationGroupName":"iOS","ParentId":3931,"label":"Toronto Catholic District School Board / Elementary / CanadianMartyrs_0251 / iOS"},{"LocationGroupId":18022,"LocationGroupName":"iOSLease","ParentId":3931,"label":"Toronto Catholic District School Board / Elementary / CanadianMartyrs_0251 / iOSLease"},{"LocationGroupId":4216,"LocationGroupName":"CardinalLeger_0419","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / CardinalLeger_0419"},{"LocationGroupId":18093,"LocationGroupName":"AppleTV","ParentId":4216,"label":"Toronto Catholic District School Board / Elementary / CardinalLeger_0419 / AppleTV"},{"LocationGroupId":4222,"LocationGroupName":"iOS","ParentId":4216,"label":"Toronto Catholic District School Board / Elementary / CardinalLeger_0419 / iOS"},{"LocationGroupId":18023,"LocationGroupName":"iOSLease","ParentId":4216,"label":"Toronto Catholic District School Board / Elementary / CardinalLeger_0419 / iOSLease"},{"LocationGroupId":3942,"LocationGroupName":"ChristTheKing_0257","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / ChristTheKing_0257"},{"LocationGroupId":18094,"LocationGroupName":"AppleTV","ParentId":3942,"label":"Toronto Catholic District School Board / Elementary / ChristTheKing_0257 / AppleTV"},{"LocationGroupId":4129,"LocationGroupName":"iOS","ParentId":3942,"label":"Toronto Catholic District School Board / Elementary / ChristTheKing_0257 / iOS"},{"LocationGroupId":18024,"LocationGroupName":"iOSLease","ParentId":3942,"label":"Toronto Catholic District School Board / Elementary / ChristTheKing_0257 / iOSLease"},{"LocationGroupId":4000,"LocationGroupName":"D\u0027arcyMcGee_0318","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / D\u0027arcyMcGee_0318"},{"LocationGroupId":18096,"LocationGroupName":"AppleTV","ParentId":4000,"label":"Toronto Catholic District School Board / Elementary / D\u0027arcyMcGee_0318 / AppleTV"},{"LocationGroupId":4130,"LocationGroupName":"iOS","ParentId":4000,"label":"Toronto Catholic District School Board / Elementary / D\u0027arcyMcGee_0318 / iOS"},{"LocationGroupId":18025,"LocationGroupName":"iOSLease","ParentId":4000,"label":"Toronto Catholic District School Board / Elementary / D\u0027arcyMcGee_0318 / iOSLease"},{"LocationGroupId":4076,"LocationGroupName":"EpiphanyofOurLordCatholicAcademy_0378","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / EpiphanyofOurLordCatholicAcademy_0378"},{"LocationGroupId":8469,"LocationGroupName":"Android","ParentId":4076,"label":"Toronto Catholic District School Board / Elementary / EpiphanyofOurLordCatholicAcademy_0378 / Android"},{"LocationGroupId":18097,"LocationGroupName":"AppleTV","ParentId":4076,"label":"Toronto Catholic District School Board / Elementary / EpiphanyofOurLordCatholicAcademy_0378 / AppleTV"},{"LocationGroupId":4132,"LocationGroupName":"iOS","ParentId":4076,"label":"Toronto Catholic District School Board / Elementary / EpiphanyofOurLordCatholicAcademy_0378 / iOS"},{"LocationGroupId":18104,"LocationGroupName":"iOSLease","ParentId":4076,"label":"Toronto Catholic District School Board / Elementary / EpiphanyofOurLordCatholicAcademy_0378 / iOSLease"},{"LocationGroupId":4151,"LocationGroupName":"FatherSerra_0391","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / FatherSerra_0391"},{"LocationGroupId":18105,"LocationGroupName":"AppleTV","ParentId":4151,"label":"Toronto Catholic District School Board / Elementary / FatherSerra_0391 / AppleTV"},{"LocationGroupId":4223,"LocationGroupName":"iOS","ParentId":4151,"label":"Toronto Catholic District School Board / Elementary / FatherSerra_0391 / iOS"},{"LocationGroupId":18106,"LocationGroupName":"iOSLease","ParentId":4151,"label":"Toronto Catholic District School Board / Elementary / FatherSerra_0391 / iOSLease"},{"LocationGroupId":6613,"LocationGroupName":"FrancisLibermannES_0324","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / FrancisLibermannES_0324"},{"LocationGroupId":18108,"LocationGroupName":"AppleTV","ParentId":6613,"label":"Toronto Catholic District School Board / Elementary / FrancisLibermannES_0324 / AppleTV"},{"LocationGroupId":6614,"LocationGroupName":"iOS","ParentId":6613,"label":"Toronto Catholic District School Board / Elementary / FrancisLibermannES_0324 / iOS"},{"LocationGroupId":18109,"LocationGroupName":"iOSLease","ParentId":6613,"label":"Toronto Catholic District School Board / Elementary / FrancisLibermannES_0324 / iOSLease"},{"LocationGroupId":4012,"LocationGroupName":"HolyAngels_0329","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / HolyAngels_0329"},{"LocationGroupId":18121,"LocationGroupName":"AppleTV","ParentId":4012,"label":"Toronto Catholic District School Board / Elementary / HolyAngels_0329 / AppleTV"},{"LocationGroupId":4131,"LocationGroupName":"iOS","ParentId":4012,"label":"Toronto Catholic District School Board / Elementary / HolyAngels_0329 / iOS"},{"LocationGroupId":18122,"LocationGroupName":"iOSLease","ParentId":4012,"label":"Toronto Catholic District School Board / Elementary / HolyAngels_0329 / iOSLease"},{"LocationGroupId":4218,"LocationGroupName":"HolyChild_0433","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / HolyChild_0433"},{"LocationGroupId":18123,"LocationGroupName":"AppleTV","ParentId":4218,"label":"Toronto Catholic District School Board / Elementary / HolyChild_0433 / AppleTV"},{"LocationGroupId":4224,"LocationGroupName":"iOS","ParentId":4218,"label":"Toronto Catholic District School Board / Elementary / HolyChild_0433 / iOS"},{"LocationGroupId":18124,"LocationGroupName":"iOSLease","ParentId":4218,"label":"Toronto Catholic District School Board / Elementary / HolyChild_0433 / iOSLease"},{"LocationGroupId":3923,"LocationGroupName":"HolyCross_0242","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / HolyCross_0242"},{"LocationGroupId":18126,"LocationGroupName":"AppleTV","ParentId":3923,"label":"Toronto Catholic District School Board / Elementary / HolyCross_0242 / AppleTV"},{"LocationGroupId":4133,"LocationGroupName":"iOS","ParentId":3923,"label":"Toronto Catholic District School Board / Elementary / HolyCross_0242 / iOS"},{"LocationGroupId":18127,"LocationGroupName":"iOSLease","ParentId":3923,"label":"Toronto Catholic District School Board / Elementary / HolyCross_0242 / iOSLease"},{"LocationGroupId":3900,"LocationGroupName":"HolyFamily_0212","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / HolyFamily_0212"},{"LocationGroupId":18129,"LocationGroupName":"AppleTV","ParentId":3900,"label":"Toronto Catholic District School Board / Elementary / HolyFamily_0212 / AppleTV"},{"LocationGroupId":4134,"LocationGroupName":"iOS","ParentId":3900,"label":"Toronto Catholic District School Board / Elementary / HolyFamily_0212 / iOS"},{"LocationGroupId":18130,"LocationGroupName":"iOSLease","ParentId":3900,"label":"Toronto Catholic District School Board / Elementary / HolyFamily_0212 / iOSLease"},{"LocationGroupId":3906,"LocationGroupName":"HolyName_0215","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / HolyName_0215"},{"LocationGroupId":18131,"LocationGroupName":"AppleTV","ParentId":3906,"label":"Toronto Catholic District School Board / Elementary / HolyName_0215 / AppleTV"},{"LocationGroupId":4135,"LocationGroupName":"iOS","ParentId":3906,"label":"Toronto Catholic District School Board / Elementary / HolyName_0215 / iOS"},{"LocationGroupId":18132,"LocationGroupName":"iOSLease","ParentId":3906,"label":"Toronto Catholic District School Board / Elementary / HolyName_0215 / iOSLease"},{"LocationGroupId":4032,"LocationGroupName":"HolyRedeemer_0353","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / HolyRedeemer_0353"},{"LocationGroupId":18134,"LocationGroupName":"AppleTV","ParentId":4032,"label":"Toronto Catholic District School Board / Elementary / HolyRedeemer_0353 / AppleTV"},{"LocationGroupId":4142,"LocationGroupName":"iOS","ParentId":4032,"label":"Toronto Catholic District School Board / Elementary / HolyRedeemer_0353 / iOS"},{"LocationGroupId":18135,"LocationGroupName":"iOSLease","ParentId":4032,"label":"Toronto Catholic District School Board / Elementary / HolyRedeemer_0353 / iOSLease"},{"LocationGroupId":3912,"LocationGroupName":"HolyRosary_0225","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / HolyRosary_0225"},{"LocationGroupId":18150,"LocationGroupName":"AppleTV","ParentId":3912,"label":"Toronto Catholic District School Board / Elementary / HolyRosary_0225 / AppleTV"},{"LocationGroupId":4143,"LocationGroupName":"iOS","ParentId":3912,"label":"Toronto Catholic District School Board / Elementary / HolyRosary_0225 / iOS"},{"LocationGroupId":18151,"LocationGroupName":"iOSLease","ParentId":3912,"label":"Toronto Catholic District School Board / Elementary / HolyRosary_0225 / iOSLease"},{"LocationGroupId":3966,"LocationGroupName":"HolySpirit_0282","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / HolySpirit_0282"},{"LocationGroupId":18152,"LocationGroupName":"AppleTV","ParentId":3966,"label":"Toronto Catholic District School Board / Elementary / HolySpirit_0282 / AppleTV"},{"LocationGroupId":4225,"LocationGroupName":"iOS","ParentId":3966,"label":"Toronto Catholic District School Board / Elementary / HolySpirit_0282 / iOS"},{"LocationGroupId":18153,"LocationGroupName":"iOSLease","ParentId":3966,"label":"Toronto Catholic District School Board / Elementary / HolySpirit_0282 / iOSLease"},{"LocationGroupId":3965,"LocationGroupName":"ImmaculateConception_0281","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / ImmaculateConception_0281"},{"LocationGroupId":18154,"LocationGroupName":"AppleTV","ParentId":3965,"label":"Toronto Catholic District School Board / Elementary / ImmaculateConception_0281 / AppleTV"},{"LocationGroupId":4226,"LocationGroupName":"iOS","ParentId":3965,"label":"Toronto Catholic District School Board / Elementary / ImmaculateConception_0281 / iOS"},{"LocationGroupId":18155,"LocationGroupName":"iOSLease","ParentId":3965,"label":"Toronto Catholic District School Board / Elementary / ImmaculateConception_0281 / iOSLease"},{"LocationGroupId":3956,"LocationGroupName":"ImmaculateHeartofMary_0272","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / ImmaculateHeartofMary_0272"},{"LocationGroupId":18156,"LocationGroupName":"AppleTV","ParentId":3956,"label":"Toronto Catholic District School Board / Elementary / ImmaculateHeartofMary_0272 / AppleTV"},{"LocationGroupId":4227,"LocationGroupName":"iOS","ParentId":3956,"label":"Toronto Catholic District School Board / Elementary / ImmaculateHeartofMary_0272 / iOS"},{"LocationGroupId":18157,"LocationGroupName":"iOSLease","ParentId":3956,"label":"Toronto Catholic District School Board / Elementary / ImmaculateHeartofMary_0272 / iOSLease"},{"LocationGroupId":4009,"LocationGroupName":"JamesCulnan_0325","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / JamesCulnan_0325"},{"LocationGroupId":18158,"LocationGroupName":"AppleTV","ParentId":4009,"label":"Toronto Catholic District School Board / Elementary / JamesCulnan_0325 / AppleTV"},{"LocationGroupId":4228,"LocationGroupName":"iOS","ParentId":4009,"label":"Toronto Catholic District School Board / Elementary / JamesCulnan_0325 / iOS"},{"LocationGroupId":18159,"LocationGroupName":"iOSLease","ParentId":4009,"label":"Toronto Catholic District School Board / Elementary / JamesCulnan_0325 / iOSLease"},{"LocationGroupId":4212,"LocationGroupName":"JosyfCardinalSlipyj_0413","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / JosyfCardinalSlipyj_0413"},{"LocationGroupId":18161,"LocationGroupName":"AppleTV","ParentId":4212,"label":"Toronto Catholic District School Board / Elementary / JosyfCardinalSlipyj_0413 / AppleTV"},{"LocationGroupId":4229,"LocationGroupName":"iOS","ParentId":4212,"label":"Toronto Catholic District School Board / Elementary / JosyfCardinalSlipyj_0413 / iOS"},{"LocationGroupId":18162,"LocationGroupName":"iOSLease","ParentId":4212,"label":"Toronto Catholic District School Board / Elementary / JosyfCardinalSlipyj_0413 / iOSLease"},{"LocationGroupId":4022,"LocationGroupName":"MotherCabrini_0340","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / MotherCabrini_0340"},{"LocationGroupId":18180,"LocationGroupName":"AppleTV","ParentId":4022,"label":"Toronto Catholic District School Board / Elementary / MotherCabrini_0340 / AppleTV"},{"LocationGroupId":4230,"LocationGroupName":"iOS","ParentId":4022,"label":"Toronto Catholic District School Board / Elementary / MotherCabrini_0340 / iOS"},{"LocationGroupId":18182,"LocationGroupName":"iOSLease","ParentId":4022,"label":"Toronto Catholic District School Board / Elementary / MotherCabrini_0340 / iOSLease"},{"LocationGroupId":4196,"LocationGroupName":"MsgrJohnCorrigan_0399","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / MsgrJohnCorrigan_0399"},{"LocationGroupId":18187,"LocationGroupName":"AppleTV","ParentId":4196,"label":"Toronto Catholic District School Board / Elementary / MsgrJohnCorrigan_0399 / AppleTV"},{"LocationGroupId":4231,"LocationGroupName":"iOS","ParentId":4196,"label":"Toronto Catholic District School Board / Elementary / MsgrJohnCorrigan_0399 / iOS"},{"LocationGroupId":18188,"LocationGroupName":"iOSLease","ParentId":4196,"label":"Toronto Catholic District School Board / Elementary / MsgrJohnCorrigan_0399 / iOSLease"},{"LocationGroupId":3972,"LocationGroupName":"NativityofOurLord_0288","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / NativityofOurLord_0288"},{"LocationGroupId":18189,"LocationGroupName":"AppleTV","ParentId":3972,"label":"Toronto Catholic District School Board / Elementary / NativityofOurLord_0288 / AppleTV"},{"LocationGroupId":4251,"LocationGroupName":"iOS","ParentId":3972,"label":"Toronto Catholic District School Board / Elementary / NativityofOurLord_0288 / iOS"},{"LocationGroupId":18190,"LocationGroupName":"iOSLease","ParentId":3972,"label":"Toronto Catholic District School Board / Elementary / NativityofOurLord_0288 / iOSLease"},{"LocationGroupId":3950,"LocationGroupName":"OLofAssumption_0265","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofAssumption_0265"},{"LocationGroupId":18195,"LocationGroupName":"AppleTV","ParentId":3950,"label":"Toronto Catholic District School Board / Elementary / OLofAssumption_0265 / AppleTV"},{"LocationGroupId":4252,"LocationGroupName":"iOS","ParentId":3950,"label":"Toronto Catholic District School Board / Elementary / OLofAssumption_0265 / iOS"},{"LocationGroupId":18198,"LocationGroupName":"iOSLease","ParentId":3950,"label":"Toronto Catholic District School Board / Elementary / OLofAssumption_0265 / iOSLease"},{"LocationGroupId":3937,"LocationGroupName":"OLofFatima_0252","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofFatima_0252"},{"LocationGroupId":18201,"LocationGroupName":"AppleTV","ParentId":3937,"label":"Toronto Catholic District School Board / Elementary / OLofFatima_0252 / AppleTV"},{"LocationGroupId":4253,"LocationGroupName":"iOS","ParentId":3937,"label":"Toronto Catholic District School Board / Elementary / OLofFatima_0252 / iOS"},{"LocationGroupId":18203,"LocationGroupName":"iOSLease","ParentId":3937,"label":"Toronto Catholic District School Board / Elementary / OLofFatima_0252 / iOSLease"},{"LocationGroupId":4071,"LocationGroupName":"OLofGrace_0372","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofGrace_0372"},{"LocationGroupId":18206,"LocationGroupName":"AppleTV","ParentId":4071,"label":"Toronto Catholic District School Board / Elementary / OLofGrace_0372 / AppleTV"},{"LocationGroupId":4254,"LocationGroupName":"iOS","ParentId":4071,"label":"Toronto Catholic District School Board / Elementary / OLofGrace_0372 / iOS"},{"LocationGroupId":18207,"LocationGroupName":"iOSLease","ParentId":4071,"label":"Toronto Catholic District School Board / Elementary / OLofGrace_0372 / iOSLease"},{"LocationGroupId":4024,"LocationGroupName":"OLofGuadalupe_0342","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofGuadalupe_0342"},{"LocationGroupId":18209,"LocationGroupName":"AppleTV","ParentId":4024,"label":"Toronto Catholic District School Board / Elementary / OLofGuadalupe_0342 / AppleTV"},{"LocationGroupId":4255,"LocationGroupName":"iOS","ParentId":4024,"label":"Toronto Catholic District School Board / Elementary / OLofGuadalupe_0342 / iOS"},{"LocationGroupId":18210,"LocationGroupName":"iOSLease","ParentId":4024,"label":"Toronto Catholic District School Board / Elementary / OLofGuadalupe_0342 / iOSLease"},{"LocationGroupId":3910,"LocationGroupName":"OLofLourdes_0221","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofLourdes_0221"},{"LocationGroupId":18212,"LocationGroupName":"AppleTV","ParentId":3910,"label":"Toronto Catholic District School Board / Elementary / OLofLourdes_0221 / AppleTV"},{"LocationGroupId":4256,"LocationGroupName":"iOS","ParentId":3910,"label":"Toronto Catholic District School Board / Elementary / OLofLourdes_0221 / iOS"},{"LocationGroupId":18213,"LocationGroupName":"iOSLease","ParentId":3910,"label":"Toronto Catholic District School Board / Elementary / OLofLourdes_0221 / iOSLease"},{"LocationGroupId":3938,"LocationGroupName":"OLofPeace_0253","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofPeace_0253"},{"LocationGroupId":18214,"LocationGroupName":"AppleTV","ParentId":3938,"label":"Toronto Catholic District School Board / Elementary / OLofPeace_0253 / AppleTV"},{"LocationGroupId":4257,"LocationGroupName":"iOS","ParentId":3938,"label":"Toronto Catholic District School Board / Elementary / OLofPeace_0253 / iOS"},{"LocationGroupId":18215,"LocationGroupName":"iOSLease","ParentId":3938,"label":"Toronto Catholic District School Board / Elementary / OLofPeace_0253 / iOSLease"},{"LocationGroupId":3920,"LocationGroupName":"OLofPerpetualHelp_0239","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofPerpetualHelp_0239"},{"LocationGroupId":18216,"LocationGroupName":"AppleTV","ParentId":3920,"label":"Toronto Catholic District School Board / Elementary / OLofPerpetualHelp_0239 / AppleTV"},{"LocationGroupId":4258,"LocationGroupName":"iOS","ParentId":3920,"label":"Toronto Catholic District School Board / Elementary / OLofPerpetualHelp_0239 / iOS"},{"LocationGroupId":18217,"LocationGroupName":"iOSLease","ParentId":3920,"label":"Toronto Catholic District School Board / Elementary / OLofPerpetualHelp_0239 / iOSLease"},{"LocationGroupId":3926,"LocationGroupName":"OLofSorrows_0246","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofSorrows_0246"},{"LocationGroupId":18220,"LocationGroupName":"AppleTV","ParentId":3926,"label":"Toronto Catholic District School Board / Elementary / OLofSorrows_0246 / AppleTV"},{"LocationGroupId":4340,"LocationGroupName":"iOS","ParentId":3926,"label":"Toronto Catholic District School Board / Elementary / OLofSorrows_0246 / iOS"},{"LocationGroupId":18222,"LocationGroupName":"iOSLease","ParentId":3926,"label":"Toronto Catholic District School Board / Elementary / OLofSorrows_0246 / iOSLease"},{"LocationGroupId":3927,"LocationGroupName":"OLofVictory_0247","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofVictory_0247"},{"LocationGroupId":18228,"LocationGroupName":"AppleTV","ParentId":3927,"label":"Toronto Catholic District School Board / Elementary / OLofVictory_0247 / AppleTV"},{"LocationGroupId":4341,"LocationGroupName":"iOS","ParentId":3927,"label":"Toronto Catholic District School Board / Elementary / OLofVictory_0247 / iOS"},{"LocationGroupId":18230,"LocationGroupName":"iOSLease","ParentId":3927,"label":"Toronto Catholic District School Board / Elementary / OLofVictory_0247 / iOSLease"},{"LocationGroupId":3957,"LocationGroupName":"OLofWisdom_0273","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / OLofWisdom_0273"},{"LocationGroupId":18420,"LocationGroupName":"AppleTV","ParentId":3957,"label":"Toronto Catholic District School Board / Elementary / OLofWisdom_0273 / AppleTV"},{"LocationGroupId":4342,"LocationGroupName":"iOS","ParentId":3957,"label":"Toronto Catholic District School Board / Elementary / OLofWisdom_0273 / iOS"},{"LocationGroupId":18422,"LocationGroupName":"iOSLease","ParentId":3957,"label":"Toronto Catholic District School Board / Elementary / OLofWisdom_0273 / iOSLease"},{"LocationGroupId":4028,"LocationGroupName":"PopeFrancis_0347","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / PopeFrancis_0347"},{"LocationGroupId":18425,"LocationGroupName":"AppleTV","ParentId":4028,"label":"Toronto Catholic District School Board / Elementary / PopeFrancis_0347 / AppleTV"},{"LocationGroupId":4434,"LocationGroupName":"iOS","ParentId":4028,"label":"Toronto Catholic District School Board / Elementary / PopeFrancis_0347 / iOS"},{"LocationGroupId":18428,"LocationGroupName":"iOSLease","ParentId":4028,"label":"Toronto Catholic District School Board / Elementary / PopeFrancis_0347 / iOSLease"},{"LocationGroupId":4073,"LocationGroupName":"PopePaul_0374","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / PopePaul_0374"},{"LocationGroupId":18431,"LocationGroupName":"AppleTV","ParentId":4073,"label":"Toronto Catholic District School Board / Elementary / PopePaul_0374 / AppleTV"},{"LocationGroupId":4343,"LocationGroupName":"iOS","ParentId":4073,"label":"Toronto Catholic District School Board / Elementary / PopePaul_0374 / iOS"},{"LocationGroupId":18433,"LocationGroupName":"iOSLease","ParentId":4073,"label":"Toronto Catholic District School Board / Elementary / PopePaul_0374 / iOSLease"},{"LocationGroupId":3943,"LocationGroupName":"PreciousBlood_0258","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / PreciousBlood_0258"},{"LocationGroupId":18434,"LocationGroupName":"AppleTV","ParentId":3943,"label":"Toronto Catholic District School Board / Elementary / PreciousBlood_0258 / AppleTV"},{"LocationGroupId":4344,"LocationGroupName":"iOS","ParentId":3943,"label":"Toronto Catholic District School Board / Elementary / PreciousBlood_0258 / iOS"},{"LocationGroupId":18435,"LocationGroupName":"iOSLease","ParentId":3943,"label":"Toronto Catholic District School Board / Elementary / PreciousBlood_0258 / iOSLease"},{"LocationGroupId":4215,"LocationGroupName":"PrinceofPeace_0416","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / PrinceofPeace_0416"},{"LocationGroupId":18439,"LocationGroupName":"AppleTV","ParentId":4215,"label":"Toronto Catholic District School Board / Elementary / PrinceofPeace_0416 / AppleTV"},{"LocationGroupId":4345,"LocationGroupName":"iOS","ParentId":4215,"label":"Toronto Catholic District School Board / Elementary / PrinceofPeace_0416 / iOS"},{"LocationGroupId":18440,"LocationGroupName":"iOSLease","ParentId":4215,"label":"Toronto Catholic District School Board / Elementary / PrinceofPeace_0416 / iOSLease"},{"LocationGroupId":4062,"LocationGroupName":"ReginaMundi_0362","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / ReginaMundi_0362"},{"LocationGroupId":18442,"LocationGroupName":"AppleTV","ParentId":4062,"label":"Toronto Catholic District School Board / Elementary / ReginaMundi_0362 / AppleTV"},{"LocationGroupId":4346,"LocationGroupName":"iOS","ParentId":4062,"label":"Toronto Catholic District School Board / Elementary / ReginaMundi_0362 / iOS"},{"LocationGroupId":18443,"LocationGroupName":"iOSLease","ParentId":4062,"label":"Toronto Catholic District School Board / Elementary / ReginaMundi_0362 / iOSLease"},{"LocationGroupId":4214,"LocationGroupName":"SacredHeart_0415","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / SacredHeart_0415"},{"LocationGroupId":18516,"LocationGroupName":"AppleTV","ParentId":4214,"label":"Toronto Catholic District School Board / Elementary / SacredHeart_0415 / AppleTV"},{"LocationGroupId":4347,"LocationGroupName":"iOS","ParentId":4214,"label":"Toronto Catholic District School Board / Elementary / SacredHeart_0415 / iOS"},{"LocationGroupId":18517,"LocationGroupName":"iOSLease","ParentId":4214,"label":"Toronto Catholic District School Board / Elementary / SacredHeart_0415 / iOSLease"},{"LocationGroupId":4068,"LocationGroupName":"SantaMaria_0368","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / SantaMaria_0368"},{"LocationGroupId":18519,"LocationGroupName":"AppleTV","ParentId":4068,"label":"Toronto Catholic District School Board / Elementary / SantaMaria_0368 / AppleTV"},{"LocationGroupId":4348,"LocationGroupName":"iOS","ParentId":4068,"label":"Toronto Catholic District School Board / Elementary / SantaMaria_0368 / iOS"},{"LocationGroupId":18520,"LocationGroupName":"iOSLease","ParentId":4068,"label":"Toronto Catholic District School Board / Elementary / SantaMaria_0368 / iOSLease"},{"LocationGroupId":4067,"LocationGroupName":"SenhorSantoCristo_0367","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / SenhorSantoCristo_0367"},{"LocationGroupId":18521,"LocationGroupName":"AppleTV","ParentId":4067,"label":"Toronto Catholic District School Board / Elementary / SenhorSantoCristo_0367 / AppleTV"},{"LocationGroupId":4349,"LocationGroupName":"iOS","ParentId":4067,"label":"Toronto Catholic District School Board / Elementary / SenhorSantoCristo_0367 / iOS"},{"LocationGroupId":18524,"LocationGroupName":"iOSLease","ParentId":4067,"label":"Toronto Catholic District School Board / Elementary / SenhorSantoCristo_0367 / iOSLease"},{"LocationGroupId":3987,"LocationGroupName":"StAgatha_0302","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAgatha_0302"},{"LocationGroupId":18525,"LocationGroupName":"AppleTV","ParentId":3987,"label":"Toronto Catholic District School Board / Elementary / StAgatha_0302 / AppleTV"},{"LocationGroupId":4350,"LocationGroupName":"iOS","ParentId":3987,"label":"Toronto Catholic District School Board / Elementary / StAgatha_0302 / iOS"},{"LocationGroupId":18526,"LocationGroupName":"iOSLease","ParentId":3987,"label":"Toronto Catholic District School Board / Elementary / StAgatha_0302 / iOSLease"},{"LocationGroupId":4026,"LocationGroupName":"StAgnes_0345","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAgnes_0345"},{"LocationGroupId":18528,"LocationGroupName":"AppleTV","ParentId":4026,"label":"Toronto Catholic District School Board / Elementary / StAgnes_0345 / AppleTV"},{"LocationGroupId":4351,"LocationGroupName":"iOS","ParentId":4026,"label":"Toronto Catholic District School Board / Elementary / StAgnes_0345 / iOS"},{"LocationGroupId":18529,"LocationGroupName":"iOSLease","ParentId":4026,"label":"Toronto Catholic District School Board / Elementary / StAgnes_0345 / iOSLease"},{"LocationGroupId":4030,"LocationGroupName":"StAidan_0350","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAidan_0350"},{"LocationGroupId":18531,"LocationGroupName":"AppleTV","ParentId":4030,"label":"Toronto Catholic District School Board / Elementary / StAidan_0350 / AppleTV"},{"LocationGroupId":4352,"LocationGroupName":"iOS","ParentId":4030,"label":"Toronto Catholic District School Board / Elementary / StAidan_0350 / iOS"},{"LocationGroupId":18532,"LocationGroupName":"iOSLease","ParentId":4030,"label":"Toronto Catholic District School Board / Elementary / StAidan_0350 / iOSLease"},{"LocationGroupId":4017,"LocationGroupName":"StAlbert_0335","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAlbert_0335"},{"LocationGroupId":18545,"LocationGroupName":"AppleTV","ParentId":4017,"label":"Toronto Catholic District School Board / Elementary / StAlbert_0335 / AppleTV"},{"LocationGroupId":4353,"LocationGroupName":"iOS","ParentId":4017,"label":"Toronto Catholic District School Board / Elementary / StAlbert_0335 / iOS"},{"LocationGroupId":18546,"LocationGroupName":"iOSLease","ParentId":4017,"label":"Toronto Catholic District School Board / Elementary / StAlbert_0335 / iOSLease"},{"LocationGroupId":4010,"LocationGroupName":"StAlphonsus_0326","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAlphonsus_0326"},{"LocationGroupId":18547,"LocationGroupName":"AppleTV","ParentId":4010,"label":"Toronto Catholic District School Board / Elementary / StAlphonsus_0326 / AppleTV"},{"LocationGroupId":4354,"LocationGroupName":"iOS","ParentId":4010,"label":"Toronto Catholic District School Board / Elementary / StAlphonsus_0326 / iOS"},{"LocationGroupId":18548,"LocationGroupName":"iOSLease","ParentId":4010,"label":"Toronto Catholic District School Board / Elementary / StAlphonsus_0326 / iOSLease"},{"LocationGroupId":3944,"LocationGroupName":"StAmbrose_0259","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAmbrose_0259"},{"LocationGroupId":18596,"LocationGroupName":"AppleTV","ParentId":3944,"label":"Toronto Catholic District School Board / Elementary / StAmbrose_0259 / AppleTV"},{"LocationGroupId":4355,"LocationGroupName":"iOS","ParentId":3944,"label":"Toronto Catholic District School Board / Elementary / StAmbrose_0259 / iOS"},{"LocationGroupId":18598,"LocationGroupName":"iOSLease","ParentId":3944,"label":"Toronto Catholic District School Board / Elementary / StAmbrose_0259 / iOSLease"},{"LocationGroupId":7823,"LocationGroupName":"StAndre_0410","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAndre_0410"},{"LocationGroupId":18599,"LocationGroupName":"AppleTV","ParentId":7823,"label":"Toronto Catholic District School Board / Elementary / StAndre_0410 / AppleTV"},{"LocationGroupId":7824,"LocationGroupName":"iOS","ParentId":7823,"label":"Toronto Catholic District School Board / Elementary / StAndre_0410 / iOS"},{"LocationGroupId":18600,"LocationGroupName":"iOSLease","ParentId":7823,"label":"Toronto Catholic District School Board / Elementary / StAndre_0410 / iOSLease"},{"LocationGroupId":3979,"LocationGroupName":"StAndrew_0295","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAndrew_0295"},{"LocationGroupId":18602,"LocationGroupName":"AppleTV","ParentId":3979,"label":"Toronto Catholic District School Board / Elementary / StAndrew_0295 / AppleTV"},{"LocationGroupId":4356,"LocationGroupName":"iOS","ParentId":3979,"label":"Toronto Catholic District School Board / Elementary / StAndrew_0295 / iOS"},{"LocationGroupId":18603,"LocationGroupName":"iOSLease","ParentId":3979,"label":"Toronto Catholic District School Board / Elementary / StAndrew_0295 / iOSLease"},{"LocationGroupId":4061,"LocationGroupName":"StAngela_0361","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAngela_0361"},{"LocationGroupId":18605,"LocationGroupName":"AppleTV","ParentId":4061,"label":"Toronto Catholic District School Board / Elementary / StAngela_0361 / AppleTV"},{"LocationGroupId":4357,"LocationGroupName":"iOS","ParentId":4061,"label":"Toronto Catholic District School Board / Elementary / StAngela_0361 / iOS"},{"LocationGroupId":18606,"LocationGroupName":"iOSLease","ParentId":4061,"label":"Toronto Catholic District School Board / Elementary / StAngela_0361 / iOSLease"},{"LocationGroupId":3924,"LocationGroupName":"StAnselm_0243","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAnselm_0243"},{"LocationGroupId":18608,"LocationGroupName":"AppleTV","ParentId":3924,"label":"Toronto Catholic District School Board / Elementary / StAnselm_0243 / AppleTV"},{"LocationGroupId":4358,"LocationGroupName":"iOS","ParentId":3924,"label":"Toronto Catholic District School Board / Elementary / StAnselm_0243 / iOS"},{"LocationGroupId":18609,"LocationGroupName":"iOSLease","ParentId":3924,"label":"Toronto Catholic District School Board / Elementary / StAnselm_0243 / iOSLease"},{"LocationGroupId":3899,"LocationGroupName":"StAnthony_0210","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAnthony_0210"},{"LocationGroupId":18612,"LocationGroupName":"AppleTV","ParentId":3899,"label":"Toronto Catholic District School Board / Elementary / StAnthony_0210 / AppleTV"},{"LocationGroupId":4359,"LocationGroupName":"iOS","ParentId":3899,"label":"Toronto Catholic District School Board / Elementary / StAnthony_0210 / iOS"},{"LocationGroupId":18613,"LocationGroupName":"iOSLease","ParentId":3899,"label":"Toronto Catholic District School Board / Elementary / StAnthony_0210 / iOSLease"},{"LocationGroupId":4021,"LocationGroupName":"StAntoineDaniel_0339","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAntoineDaniel_0339"},{"LocationGroupId":18615,"LocationGroupName":"AppleTV","ParentId":4021,"label":"Toronto Catholic District School Board / Elementary / StAntoineDaniel_0339 / AppleTV"},{"LocationGroupId":4360,"LocationGroupName":"iOS","ParentId":4021,"label":"Toronto Catholic District School Board / Elementary / StAntoineDaniel_0339 / iOS"},{"LocationGroupId":18616,"LocationGroupName":"iOSLease","ParentId":4021,"label":"Toronto Catholic District School Board / Elementary / StAntoineDaniel_0339 / iOSLease"},{"LocationGroupId":4011,"LocationGroupName":"StAugustine_0328","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StAugustine_0328"},{"LocationGroupId":18618,"LocationGroupName":"AppleTV","ParentId":4011,"label":"Toronto Catholic District School Board / Elementary / StAugustine_0328 / AppleTV"},{"LocationGroupId":4361,"LocationGroupName":"iOS","ParentId":4011,"label":"Toronto Catholic District School Board / Elementary / StAugustine_0328 / iOS"},{"LocationGroupId":18619,"LocationGroupName":"iOSLease","ParentId":4011,"label":"Toronto Catholic District School Board / Elementary / StAugustine_0328 / iOSLease"},{"LocationGroupId":3958,"LocationGroupName":"StBarbara_0274","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBarbara_0274"},{"LocationGroupId":18620,"LocationGroupName":"AppleTV","ParentId":3958,"label":"Toronto Catholic District School Board / Elementary / StBarbara_0274 / AppleTV"},{"LocationGroupId":4362,"LocationGroupName":"iOS","ParentId":3958,"label":"Toronto Catholic District School Board / Elementary / StBarbara_0274 / iOS"},{"LocationGroupId":18622,"LocationGroupName":"iOSLease","ParentId":3958,"label":"Toronto Catholic District School Board / Elementary / StBarbara_0274 / iOSLease"},{"LocationGroupId":4031,"LocationGroupName":"StBarnabas_0351","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBarnabas_0351"},{"LocationGroupId":18623,"LocationGroupName":"AppleTV","ParentId":4031,"label":"Toronto Catholic District School Board / Elementary / StBarnabas_0351 / AppleTV"},{"LocationGroupId":4363,"LocationGroupName":"iOS","ParentId":4031,"label":"Toronto Catholic District School Board / Elementary / StBarnabas_0351 / iOS"},{"LocationGroupId":18625,"LocationGroupName":"iOSLease","ParentId":4031,"label":"Toronto Catholic District School Board / Elementary / StBarnabas_0351 / iOSLease"},{"LocationGroupId":4016,"LocationGroupName":"StBartholomew_0334","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBartholomew_0334"},{"LocationGroupId":18626,"LocationGroupName":"AppleTV","ParentId":4016,"label":"Toronto Catholic District School Board / Elementary / StBartholomew_0334 / AppleTV"},{"LocationGroupId":4364,"LocationGroupName":"iOS","ParentId":4016,"label":"Toronto Catholic District School Board / Elementary / StBartholomew_0334 / iOS"},{"LocationGroupId":18627,"LocationGroupName":"iOSLease","ParentId":4016,"label":"Toronto Catholic District School Board / Elementary / StBartholomew_0334 / iOSLease"},{"LocationGroupId":4217,"LocationGroupName":"StBede_0425","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBede_0425"},{"LocationGroupId":18629,"LocationGroupName":"AppleTV","ParentId":4217,"label":"Toronto Catholic District School Board / Elementary / StBede_0425 / AppleTV"},{"LocationGroupId":4365,"LocationGroupName":"iOS","ParentId":4217,"label":"Toronto Catholic District School Board / Elementary / StBede_0425 / iOS"},{"LocationGroupId":18630,"LocationGroupName":"iOSLease","ParentId":4217,"label":"Toronto Catholic District School Board / Elementary / StBede_0425 / iOSLease"},{"LocationGroupId":3952,"LocationGroupName":"StBenedict_0267","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBenedict_0267"},{"LocationGroupId":18632,"LocationGroupName":"AppleTV","ParentId":3952,"label":"Toronto Catholic District School Board / Elementary / StBenedict_0267 / AppleTV"},{"LocationGroupId":4366,"LocationGroupName":"iOS","ParentId":3952,"label":"Toronto Catholic District School Board / Elementary / StBenedict_0267 / iOS"},{"LocationGroupId":18633,"LocationGroupName":"iOSLease","ParentId":3952,"label":"Toronto Catholic District School Board / Elementary / StBenedict_0267 / iOSLease"},{"LocationGroupId":3946,"LocationGroupName":"StBernard_0261","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBernard_0261"},{"LocationGroupId":18634,"LocationGroupName":"AppleTV","ParentId":3946,"label":"Toronto Catholic District School Board / Elementary / StBernard_0261 / AppleTV"},{"LocationGroupId":4367,"LocationGroupName":"iOS","ParentId":3946,"label":"Toronto Catholic District School Board / Elementary / StBernard_0261 / iOS"},{"LocationGroupId":18636,"LocationGroupName":"iOSLease","ParentId":3946,"label":"Toronto Catholic District School Board / Elementary / StBernard_0261 / iOSLease"},{"LocationGroupId":3948,"LocationGroupName":"StBonaventure_0263","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBonaventure_0263"},{"LocationGroupId":18637,"LocationGroupName":"AppleTV","ParentId":3948,"label":"Toronto Catholic District School Board / Elementary / StBonaventure_0263 / AppleTV"},{"LocationGroupId":4368,"LocationGroupName":"iOS","ParentId":3948,"label":"Toronto Catholic District School Board / Elementary / StBonaventure_0263 / iOS"},{"LocationGroupId":18638,"LocationGroupName":"iOSLease","ParentId":3948,"label":"Toronto Catholic District School Board / Elementary / StBonaventure_0263 / iOSLease"},{"LocationGroupId":3961,"LocationGroupName":"StBoniface_0277","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBoniface_0277"},{"LocationGroupId":18641,"LocationGroupName":"AppleTV","ParentId":3961,"label":"Toronto Catholic District School Board / Elementary / StBoniface_0277 / AppleTV"},{"LocationGroupId":4369,"LocationGroupName":"iOS","ParentId":3961,"label":"Toronto Catholic District School Board / Elementary / StBoniface_0277 / iOS"},{"LocationGroupId":18642,"LocationGroupName":"iOSLease","ParentId":3961,"label":"Toronto Catholic District School Board / Elementary / StBoniface_0277 / iOSLease"},{"LocationGroupId":4060,"LocationGroupName":"StBrendan_0359","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBrendan_0359"},{"LocationGroupId":18644,"LocationGroupName":"AppleTV","ParentId":4060,"label":"Toronto Catholic District School Board / Elementary / StBrendan_0359 / AppleTV"},{"LocationGroupId":4370,"LocationGroupName":"iOS","ParentId":4060,"label":"Toronto Catholic District School Board / Elementary / StBrendan_0359 / iOS"},{"LocationGroupId":18645,"LocationGroupName":"iOSLease","ParentId":4060,"label":"Toronto Catholic District School Board / Elementary / StBrendan_0359 / iOSLease"},{"LocationGroupId":3917,"LocationGroupName":"StBrigid_0235","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBrigid_0235"},{"LocationGroupId":18646,"LocationGroupName":"AppleTV","ParentId":3917,"label":"Toronto Catholic District School Board / Elementary / StBrigid_0235 / AppleTV"},{"LocationGroupId":4371,"LocationGroupName":"iOS","ParentId":3917,"label":"Toronto Catholic District School Board / Elementary / StBrigid_0235 / iOS"},{"LocationGroupId":18647,"LocationGroupName":"iOSLease","ParentId":3917,"label":"Toronto Catholic District School Board / Elementary / StBrigid_0235 / iOSLease"},{"LocationGroupId":4074,"LocationGroupName":"StBruno_0376","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StBruno_0376"},{"LocationGroupId":18653,"LocationGroupName":"AppleTV","ParentId":4074,"label":"Toronto Catholic District School Board / Elementary / StBruno_0376 / AppleTV"},{"LocationGroupId":4372,"LocationGroupName":"iOS","ParentId":4074,"label":"Toronto Catholic District School Board / Elementary / StBruno_0376 / iOS"},{"LocationGroupId":18654,"LocationGroupName":"iOSLease","ParentId":4074,"label":"Toronto Catholic District School Board / Elementary / StBruno_0376 / iOSLease"},{"LocationGroupId":3999,"LocationGroupName":"StCatherine_0316","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StCatherine_0316"},{"LocationGroupId":18655,"LocationGroupName":"AppleTV","ParentId":3999,"label":"Toronto Catholic District School Board / Elementary / StCatherine_0316 / AppleTV"},{"LocationGroupId":4373,"LocationGroupName":"iOS","ParentId":3999,"label":"Toronto Catholic District School Board / Elementary / StCatherine_0316 / iOS"},{"LocationGroupId":18656,"LocationGroupName":"iOSLease","ParentId":3999,"label":"Toronto Catholic District School Board / Elementary / StCatherine_0316 / iOSLease"},{"LocationGroupId":3908,"LocationGroupName":"StCecilia_0218","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StCecilia_0218"},{"LocationGroupId":18658,"LocationGroupName":"AppleTV","ParentId":3908,"label":"Toronto Catholic District School Board / Elementary / StCecilia_0218 / AppleTV"},{"LocationGroupId":4374,"LocationGroupName":"iOS","ParentId":3908,"label":"Toronto Catholic District School Board / Elementary / StCecilia_0218 / iOS"},{"LocationGroupId":18659,"LocationGroupName":"iOSLease","ParentId":3908,"label":"Toronto Catholic District School Board / Elementary / StCecilia_0218 / iOSLease"},{"LocationGroupId":3930,"LocationGroupName":"StCharles_0250","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StCharles_0250"},{"LocationGroupId":18661,"LocationGroupName":"AppleTV","ParentId":3930,"label":"Toronto Catholic District School Board / Elementary / StCharles_0250 / AppleTV"},{"LocationGroupId":4375,"LocationGroupName":"iOS","ParentId":3930,"label":"Toronto Catholic District School Board / Elementary / StCharles_0250 / iOS"},{"LocationGroupId":18662,"LocationGroupName":"iOSLease","ParentId":3930,"label":"Toronto Catholic District School Board / Elementary / StCharles_0250 / iOSLease"},{"LocationGroupId":4063,"LocationGroupName":"StCharlesGarnier_0363","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StCharlesGarnier_0363"},{"LocationGroupId":18664,"LocationGroupName":"AppleTV","ParentId":4063,"label":"Toronto Catholic District School Board / Elementary / StCharlesGarnier_0363 / AppleTV"},{"LocationGroupId":4376,"LocationGroupName":"iOS","ParentId":4063,"label":"Toronto Catholic District School Board / Elementary / StCharlesGarnier_0363 / iOS"},{"LocationGroupId":18665,"LocationGroupName":"iOSLease","ParentId":4063,"label":"Toronto Catholic District School Board / Elementary / StCharlesGarnier_0363 / iOSLease"},{"LocationGroupId":3905,"LocationGroupName":"StClare_0214","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StClare_0214"},{"LocationGroupId":18667,"LocationGroupName":"AppleTV","ParentId":3905,"label":"Toronto Catholic District School Board / Elementary / StClare_0214 / AppleTV"},{"LocationGroupId":4377,"LocationGroupName":"iOS","ParentId":3905,"label":"Toronto Catholic District School Board / Elementary / StClare_0214 / iOS"},{"LocationGroupId":18668,"LocationGroupName":"iOSLease","ParentId":3905,"label":"Toronto Catholic District School Board / Elementary / StClare_0214 / iOSLease"},{"LocationGroupId":3992,"LocationGroupName":"StClement_0307","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StClement_0307"},{"LocationGroupId":18669,"LocationGroupName":"AppleTV","ParentId":3992,"label":"Toronto Catholic District School Board / Elementary / StClement_0307 / AppleTV"},{"LocationGroupId":4378,"LocationGroupName":"iOS","ParentId":3992,"label":"Toronto Catholic District School Board / Elementary / StClement_0307 / iOS"},{"LocationGroupId":18670,"LocationGroupName":"iOSLease","ParentId":3992,"label":"Toronto Catholic District School Board / Elementary / StClement_0307 / iOSLease"},{"LocationGroupId":4078,"LocationGroupName":"StColumba_0380","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StColumba_0380"},{"LocationGroupId":18678,"LocationGroupName":"AppleTV","ParentId":4078,"label":"Toronto Catholic District School Board / Elementary / StColumba_0380 / AppleTV"},{"LocationGroupId":4379,"LocationGroupName":"iOS","ParentId":4078,"label":"Toronto Catholic District School Board / Elementary / StColumba_0380 / iOS"},{"LocationGroupId":18679,"LocationGroupName":"iOSLease","ParentId":4078,"label":"Toronto Catholic District School Board / Elementary / StColumba_0380 / iOSLease"},{"LocationGroupId":4193,"LocationGroupName":"StConrad_0396","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StConrad_0396"},{"LocationGroupId":18680,"LocationGroupName":"AppleTV","ParentId":4193,"label":"Toronto Catholic District School Board / Elementary / StConrad_0396 / AppleTV"},{"LocationGroupId":4380,"LocationGroupName":"iOS","ParentId":4193,"label":"Toronto Catholic District School Board / Elementary / StConrad_0396 / iOS"},{"LocationGroupId":18682,"LocationGroupName":"iOSLease","ParentId":4193,"label":"Toronto Catholic District School Board / Elementary / StConrad_0396 / iOSLease"},{"LocationGroupId":4195,"LocationGroupName":"StCosmasandDamian_0398","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StCosmasandDamian_0398"},{"LocationGroupId":18683,"LocationGroupName":"AppleTV","ParentId":4195,"label":"Toronto Catholic District School Board / Elementary / StCosmasandDamian_0398 / AppleTV"},{"LocationGroupId":4381,"LocationGroupName":"iOS","ParentId":4195,"label":"Toronto Catholic District School Board / Elementary / StCosmasandDamian_0398 / iOS"},{"LocationGroupId":18684,"LocationGroupName":"iOSLease","ParentId":4195,"label":"Toronto Catholic District School Board / Elementary / StCosmasandDamian_0398 / iOSLease"},{"LocationGroupId":3954,"LocationGroupName":"StCyril_0270","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StCyril_0270"},{"LocationGroupId":18749,"LocationGroupName":"AppleTV","ParentId":3954,"label":"Toronto Catholic District School Board / Elementary / StCyril_0270 / AppleTV"},{"LocationGroupId":4382,"LocationGroupName":"iOS","ParentId":3954,"label":"Toronto Catholic District School Board / Elementary / StCyril_0270 / iOS"},{"LocationGroupId":18750,"LocationGroupName":"iOSLease","ParentId":3954,"label":"Toronto Catholic District School Board / Elementary / StCyril_0270 / iOSLease"},{"LocationGroupId":4139,"LocationGroupName":"StDemetrius_0384","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StDemetrius_0384"},{"LocationGroupId":18751,"LocationGroupName":"AppleTV","ParentId":4139,"label":"Toronto Catholic District School Board / Elementary / StDemetrius_0384 / AppleTV"},{"LocationGroupId":4383,"LocationGroupName":"iOS","ParentId":4139,"label":"Toronto Catholic District School Board / Elementary / StDemetrius_0384 / iOS"},{"LocationGroupId":18752,"LocationGroupName":"iOSLease","ParentId":4139,"label":"Toronto Catholic District School Board / Elementary / StDemetrius_0384 / iOSLease"},{"LocationGroupId":3995,"LocationGroupName":"StDenis_0310","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StDenis_0310"},{"LocationGroupId":18753,"LocationGroupName":"AppleTV","ParentId":3995,"label":"Toronto Catholic District School Board / Elementary / StDenis_0310 / AppleTV"},{"LocationGroupId":4385,"LocationGroupName":"iOS","ParentId":3995,"label":"Toronto Catholic District School Board / Elementary / StDenis_0310 / iOS"},{"LocationGroupId":18754,"LocationGroupName":"iOSLease","ParentId":3995,"label":"Toronto Catholic District School Board / Elementary / StDenis_0310 / iOSLease"},{"LocationGroupId":4219,"LocationGroupName":"StDominicSavio_0467","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StDominicSavio_0467"},{"LocationGroupId":18755,"LocationGroupName":"AppleTV","ParentId":4219,"label":"Toronto Catholic District School Board / Elementary / StDominicSavio_0467 / AppleTV"},{"LocationGroupId":4386,"LocationGroupName":"iOS","ParentId":4219,"label":"Toronto Catholic District School Board / Elementary / StDominicSavio_0467 / iOS"},{"LocationGroupId":18756,"LocationGroupName":"iOSLease","ParentId":4219,"label":"Toronto Catholic District School Board / Elementary / StDominicSavio_0467 / iOSLease"},{"LocationGroupId":4023,"LocationGroupName":"StDorothy_0341","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StDorothy_0341"},{"LocationGroupId":18757,"LocationGroupName":"AppleTV","ParentId":4023,"label":"Toronto Catholic District School Board / Elementary / StDorothy_0341 / AppleTV"},{"LocationGroupId":4387,"LocationGroupName":"iOS","ParentId":4023,"label":"Toronto Catholic District School Board / Elementary / StDorothy_0341 / iOS"},{"LocationGroupId":18758,"LocationGroupName":"iOSLease","ParentId":4023,"label":"Toronto Catholic District School Board / Elementary / StDorothy_0341 / iOSLease"},{"LocationGroupId":3922,"LocationGroupName":"StDunstan_0241","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StDunstan_0241"},{"LocationGroupId":18759,"LocationGroupName":"AppleTV","ParentId":3922,"label":"Toronto Catholic District School Board / Elementary / StDunstan_0241 / AppleTV"},{"LocationGroupId":4388,"LocationGroupName":"iOS","ParentId":3922,"label":"Toronto Catholic District School Board / Elementary / StDunstan_0241 / iOS"},{"LocationGroupId":18760,"LocationGroupName":"iOSLease","ParentId":3922,"label":"Toronto Catholic District School Board / Elementary / StDunstan_0241 / iOSLease"},{"LocationGroupId":4057,"LocationGroupName":"StEdmundCampion_0356","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StEdmundCampion_0356"},{"LocationGroupId":18761,"LocationGroupName":"AppleTV","ParentId":4057,"label":"Toronto Catholic District School Board / Elementary / StEdmundCampion_0356 / AppleTV"},{"LocationGroupId":4389,"LocationGroupName":"iOS","ParentId":4057,"label":"Toronto Catholic District School Board / Elementary / StEdmundCampion_0356 / iOS"},{"LocationGroupId":18762,"LocationGroupName":"iOSLease","ParentId":4057,"label":"Toronto Catholic District School Board / Elementary / StEdmundCampion_0356 / iOSLease"},{"LocationGroupId":3940,"LocationGroupName":"StEdward_0255","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StEdward_0255"},{"LocationGroupId":18763,"LocationGroupName":"AppleTV","ParentId":3940,"label":"Toronto Catholic District School Board / Elementary / StEdward_0255 / AppleTV"},{"LocationGroupId":4390,"LocationGroupName":"iOS","ParentId":3940,"label":"Toronto Catholic District School Board / Elementary / StEdward_0255 / iOS"},{"LocationGroupId":18764,"LocationGroupName":"iOSLease","ParentId":3940,"label":"Toronto Catholic District School Board / Elementary / StEdward_0255 / iOSLease"},{"LocationGroupId":3963,"LocationGroupName":"StElizabeth_0278","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StElizabeth_0278"},{"LocationGroupId":18765,"LocationGroupName":"AppleTV","ParentId":3963,"label":"Toronto Catholic District School Board / Elementary / StElizabeth_0278 / AppleTV"},{"LocationGroupId":4391,"LocationGroupName":"iOS","ParentId":3963,"label":"Toronto Catholic District School Board / Elementary / StElizabeth_0278 / iOS"},{"LocationGroupId":18766,"LocationGroupName":"iOSLease","ParentId":3963,"label":"Toronto Catholic District School Board / Elementary / StElizabeth_0278 / iOSLease"},{"LocationGroupId":4141,"LocationGroupName":"StElizabethSeton_0386","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StElizabethSeton_0386"},{"LocationGroupId":18767,"LocationGroupName":"AppleTV","ParentId":4141,"label":"Toronto Catholic District School Board / Elementary / StElizabethSeton_0386 / AppleTV"},{"LocationGroupId":4392,"LocationGroupName":"iOS","ParentId":4141,"label":"Toronto Catholic District School Board / Elementary / StElizabethSeton_0386 / iOS"},{"LocationGroupId":18768,"LocationGroupName":"iOSLease","ParentId":4141,"label":"Toronto Catholic District School Board / Elementary / StElizabethSeton_0386 / iOSLease"},{"LocationGroupId":4070,"LocationGroupName":"StellaMaris_0371","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StellaMaris_0371"},{"LocationGroupId":18769,"LocationGroupName":"AppleTV","ParentId":4070,"label":"Toronto Catholic District School Board / Elementary / StellaMaris_0371 / AppleTV"},{"LocationGroupId":4393,"LocationGroupName":"iOS","ParentId":4070,"label":"Toronto Catholic District School Board / Elementary / StellaMaris_0371 / iOS"},{"LocationGroupId":18770,"LocationGroupName":"iOSLease","ParentId":4070,"label":"Toronto Catholic District School Board / Elementary / StellaMaris_0371 / iOSLease"},{"LocationGroupId":4019,"LocationGroupName":"StEugene_0337","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StEugene_0337"},{"LocationGroupId":18771,"LocationGroupName":"AppleTV","ParentId":4019,"label":"Toronto Catholic District School Board / Elementary / StEugene_0337 / AppleTV"},{"LocationGroupId":4394,"LocationGroupName":"iOS","ParentId":4019,"label":"Toronto Catholic District School Board / Elementary / StEugene_0337 / iOS"},{"LocationGroupId":18772,"LocationGroupName":"iOSLease","ParentId":4019,"label":"Toronto Catholic District School Board / Elementary / StEugene_0337 / iOSLease"},{"LocationGroupId":4059,"LocationGroupName":"StFidelis_0358","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StFidelis_0358"},{"LocationGroupId":18773,"LocationGroupName":"AppleTV","ParentId":4059,"label":"Toronto Catholic District School Board / Elementary / StFidelis_0358 / AppleTV"},{"LocationGroupId":4395,"LocationGroupName":"iOS","ParentId":4059,"label":"Toronto Catholic District School Board / Elementary / StFidelis_0358 / iOS"},{"LocationGroupId":18774,"LocationGroupName":"iOSLease","ParentId":4059,"label":"Toronto Catholic District School Board / Elementary / StFidelis_0358 / iOSLease"},{"LocationGroupId":4140,"LocationGroupName":"StFlorence_0385","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StFlorence_0385"},{"LocationGroupId":18775,"LocationGroupName":"AppleTV","ParentId":4140,"label":"Toronto Catholic District School Board / Elementary / StFlorence_0385 / AppleTV"},{"LocationGroupId":4396,"LocationGroupName":"iOS","ParentId":4140,"label":"Toronto Catholic District School Board / Elementary / StFlorence_0385 / iOS"},{"LocationGroupId":18776,"LocationGroupName":"iOSLease","ParentId":4140,"label":"Toronto Catholic District School Board / Elementary / StFlorence_0385 / iOSLease"},{"LocationGroupId":3325,"LocationGroupName":"StFrancisAssisi_0206","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StFrancisAssisi_0206"},{"LocationGroupId":18777,"LocationGroupName":"AppleTV","ParentId":3325,"label":"Toronto Catholic District School Board / Elementary / StFrancisAssisi_0206 / AppleTV"},{"LocationGroupId":4397,"LocationGroupName":"iOS","ParentId":3325,"label":"Toronto Catholic District School Board / Elementary / StFrancisAssisi_0206 / iOS"},{"LocationGroupId":18779,"LocationGroupName":"iOSLease","ParentId":3325,"label":"Toronto Catholic District School Board / Elementary / StFrancisAssisi_0206 / iOSLease"},{"LocationGroupId":4066,"LocationGroupName":"StFrancisdeSales_0366","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StFrancisdeSales_0366"},{"LocationGroupId":18780,"LocationGroupName":"AppleTV","ParentId":4066,"label":"Toronto Catholic District School Board / Elementary / StFrancisdeSales_0366 / AppleTV"},{"LocationGroupId":4398,"LocationGroupName":"iOS","ParentId":4066,"label":"Toronto Catholic District School Board / Elementary / StFrancisdeSales_0366 / iOS"},{"LocationGroupId":18781,"LocationGroupName":"iOSLease","ParentId":4066,"label":"Toronto Catholic District School Board / Elementary / StFrancisdeSales_0366 / iOSLease"},{"LocationGroupId":3955,"LocationGroupName":"StFrancisXavier_0271","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StFrancisXavier_0271"},{"LocationGroupId":18782,"LocationGroupName":"AppleTV","ParentId":3955,"label":"Toronto Catholic District School Board / Elementary / StFrancisXavier_0271 / AppleTV"},{"LocationGroupId":4399,"LocationGroupName":"iOS","ParentId":3955,"label":"Toronto Catholic District School Board / Elementary / StFrancisXavier_0271 / iOS"},{"LocationGroupId":18783,"LocationGroupName":"iOSLease","ParentId":3955,"label":"Toronto Catholic District School Board / Elementary / StFrancisXavier_0271 / iOSLease"},{"LocationGroupId":3941,"LocationGroupName":"StGabriel_0256","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StGabriel_0256"},{"LocationGroupId":18784,"LocationGroupName":"AppleTV","ParentId":3941,"label":"Toronto Catholic District School Board / Elementary / StGabriel_0256 / AppleTV"},{"LocationGroupId":4400,"LocationGroupName":"iOS","ParentId":3941,"label":"Toronto Catholic District School Board / Elementary / StGabriel_0256 / iOS"},{"LocationGroupId":18785,"LocationGroupName":"iOSLease","ParentId":3941,"label":"Toronto Catholic District School Board / Elementary / StGabriel_0256 / iOSLease"},{"LocationGroupId":4075,"LocationGroupName":"StGabrielLalemant_0377","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StGabrielLalemant_0377"},{"LocationGroupId":18786,"LocationGroupName":"AppleTV","ParentId":4075,"label":"Toronto Catholic District School Board / Elementary / StGabrielLalemant_0377 / AppleTV"},{"LocationGroupId":4401,"LocationGroupName":"iOS","ParentId":4075,"label":"Toronto Catholic District School Board / Elementary / StGabrielLalemant_0377 / iOS"},{"LocationGroupId":18787,"LocationGroupName":"iOSLease","ParentId":4075,"label":"Toronto Catholic District School Board / Elementary / StGabrielLalemant_0377 / iOSLease"},{"LocationGroupId":4001,"LocationGroupName":"StGerald_0319","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StGerald_0319"},{"LocationGroupId":18788,"LocationGroupName":"AppleTV","ParentId":4001,"label":"Toronto Catholic District School Board / Elementary / StGerald_0319 / AppleTV"},{"LocationGroupId":4402,"LocationGroupName":"iOS","ParentId":4001,"label":"Toronto Catholic District School Board / Elementary / StGerald_0319 / iOS"},{"LocationGroupId":18789,"LocationGroupName":"iOSLease","ParentId":4001,"label":"Toronto Catholic District School Board / Elementary / StGerald_0319 / iOSLease"},{"LocationGroupId":3947,"LocationGroupName":"StGregory_0262","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StGregory_0262"},{"LocationGroupId":18790,"LocationGroupName":"AppleTV","ParentId":3947,"label":"Toronto Catholic District School Board / Elementary / StGregory_0262 / AppleTV"},{"LocationGroupId":4404,"LocationGroupName":"iOS","ParentId":3947,"label":"Toronto Catholic District School Board / Elementary / StGregory_0262 / iOS"},{"LocationGroupId":18791,"LocationGroupName":"iOSLease","ParentId":3947,"label":"Toronto Catholic District School Board / Elementary / StGregory_0262 / iOSLease"},{"LocationGroupId":3897,"LocationGroupName":"StHelen_0208","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StHelen_0208"},{"LocationGroupId":18792,"LocationGroupName":"AppleTV","ParentId":3897,"label":"Toronto Catholic District School Board / Elementary / StHelen_0208 / AppleTV"},{"LocationGroupId":4405,"LocationGroupName":"iOS","ParentId":3897,"label":"Toronto Catholic District School Board / Elementary / StHelen_0208 / iOS"},{"LocationGroupId":18793,"LocationGroupName":"iOSLease","ParentId":3897,"label":"Toronto Catholic District School Board / Elementary / StHelen_0208 / iOSLease"},{"LocationGroupId":4156,"LocationGroupName":"StHenry_0395","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StHenry_0395"},{"LocationGroupId":18794,"LocationGroupName":"AppleTV","ParentId":4156,"label":"Toronto Catholic District School Board / Elementary / StHenry_0395 / AppleTV"},{"LocationGroupId":4406,"LocationGroupName":"iOS","ParentId":4156,"label":"Toronto Catholic District School Board / Elementary / StHenry_0395 / iOS"},{"LocationGroupId":18795,"LocationGroupName":"iOSLease","ParentId":4156,"label":"Toronto Catholic District School Board / Elementary / StHenry_0395 / iOSLease"},{"LocationGroupId":4065,"LocationGroupName":"StIgnatiusofLoyola_0365","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StIgnatiusofLoyola_0365"},{"LocationGroupId":18796,"LocationGroupName":"AppleTV","ParentId":4065,"label":"Toronto Catholic District School Board / Elementary / StIgnatiusofLoyola_0365 / AppleTV"},{"LocationGroupId":4407,"LocationGroupName":"iOS","ParentId":4065,"label":"Toronto Catholic District School Board / Elementary / StIgnatiusofLoyola_0365 / iOS"},{"LocationGroupId":18797,"LocationGroupName":"iOSLease","ParentId":4065,"label":"Toronto Catholic District School Board / Elementary / StIgnatiusofLoyola_0365 / iOSLease"},{"LocationGroupId":3998,"LocationGroupName":"StIsaacJogues_0315","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StIsaacJogues_0315"},{"LocationGroupId":18798,"LocationGroupName":"AppleTV","ParentId":3998,"label":"Toronto Catholic District School Board / Elementary / StIsaacJogues_0315 / AppleTV"},{"LocationGroupId":4408,"LocationGroupName":"iOS","ParentId":3998,"label":"Toronto Catholic District School Board / Elementary / StIsaacJogues_0315 / iOS"},{"LocationGroupId":18799,"LocationGroupName":"iOSLease","ParentId":3998,"label":"Toronto Catholic District School Board / Elementary / StIsaacJogues_0315 / iOSLease"},{"LocationGroupId":3959,"LocationGroupName":"StJames_0275","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJames_0275"},{"LocationGroupId":18800,"LocationGroupName":"AppleTV","ParentId":3959,"label":"Toronto Catholic District School Board / Elementary / StJames_0275 / AppleTV"},{"LocationGroupId":4409,"LocationGroupName":"iOS","ParentId":3959,"label":"Toronto Catholic District School Board / Elementary / StJames_0275 / iOS"},{"LocationGroupId":18801,"LocationGroupName":"iOSLease","ParentId":3959,"label":"Toronto Catholic District School Board / Elementary / StJames_0275 / iOSLease"},{"LocationGroupId":3980,"LocationGroupName":"StJaneFrances_0296","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJaneFrances_0296"},{"LocationGroupId":18808,"LocationGroupName":"AppleTV","ParentId":3980,"label":"Toronto Catholic District School Board / Elementary / StJaneFrances_0296 / AppleTV"},{"LocationGroupId":4410,"LocationGroupName":"iOS","ParentId":3980,"label":"Toronto Catholic District School Board / Elementary / StJaneFrances_0296 / iOS"},{"LocationGroupId":18809,"LocationGroupName":"iOSLease","ParentId":3980,"label":"Toronto Catholic District School Board / Elementary / StJaneFrances_0296 / iOSLease"},{"LocationGroupId":4155,"LocationGroupName":"StJeanDeBrebeuf_0394","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJeanDeBrebeuf_0394"},{"LocationGroupId":18810,"LocationGroupName":"AppleTV","ParentId":4155,"label":"Toronto Catholic District School Board / Elementary / StJeanDeBrebeuf_0394 / AppleTV"},{"LocationGroupId":4411,"LocationGroupName":"iOS","ParentId":4155,"label":"Toronto Catholic District School Board / Elementary / StJeanDeBrebeuf_0394 / iOS"},{"LocationGroupId":18811,"LocationGroupName":"iOSLease","ParentId":4155,"label":"Toronto Catholic District School Board / Elementary / StJeanDeBrebeuf_0394 / iOSLease"},{"LocationGroupId":3978,"LocationGroupName":"StJerome_0294","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJerome_0294"},{"LocationGroupId":18812,"LocationGroupName":"AppleTV","ParentId":3978,"label":"Toronto Catholic District School Board / Elementary / StJerome_0294 / AppleTV"},{"LocationGroupId":4412,"LocationGroupName":"iOS","ParentId":3978,"label":"Toronto Catholic District School Board / Elementary / StJerome_0294 / iOS"},{"LocationGroupId":18813,"LocationGroupName":"iOSLease","ParentId":3978,"label":"Toronto Catholic District School Board / Elementary / StJerome_0294 / iOSLease"},{"LocationGroupId":3976,"LocationGroupName":"StJoachim_0292","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJoachim_0292"},{"LocationGroupId":18814,"LocationGroupName":"AppleTV","ParentId":3976,"label":"Toronto Catholic District School Board / Elementary / StJoachim_0292 / AppleTV"},{"LocationGroupId":4421,"LocationGroupName":"iOS","ParentId":3976,"label":"Toronto Catholic District School Board / Elementary / StJoachim_0292 / iOS"},{"LocationGroupId":18815,"LocationGroupName":"iOSLease","ParentId":3976,"label":"Toronto Catholic District School Board / Elementary / StJoachim_0292 / iOSLease"},{"LocationGroupId":3989,"LocationGroupName":"StJohnBosco_0305","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJohnBosco_0305"},{"LocationGroupId":18816,"LocationGroupName":"AppleTV","ParentId":3989,"label":"Toronto Catholic District School Board / Elementary / StJohnBosco_0305 / AppleTV"},{"LocationGroupId":4422,"LocationGroupName":"iOS","ParentId":3989,"label":"Toronto Catholic District School Board / Elementary / StJohnBosco_0305 / iOS"},{"LocationGroupId":18817,"LocationGroupName":"iOSLease","ParentId":3989,"label":"Toronto Catholic District School Board / Elementary / StJohnBosco_0305 / iOSLease"},{"LocationGroupId":3925,"LocationGroupName":"StJohnTheEvangelist_0244","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJohnTheEvangelist_0244"},{"LocationGroupId":18818,"LocationGroupName":"AppleTV","ParentId":3925,"label":"Toronto Catholic District School Board / Elementary / StJohnTheEvangelist_0244 / AppleTV"},{"LocationGroupId":4423,"LocationGroupName":"iOS","ParentId":3925,"label":"Toronto Catholic District School Board / Elementary / StJohnTheEvangelist_0244 / iOS"},{"LocationGroupId":18819,"LocationGroupName":"iOSLease","ParentId":3925,"label":"Toronto Catholic District School Board / Elementary / StJohnTheEvangelist_0244 / iOSLease"},{"LocationGroupId":3909,"LocationGroupName":"StJohnToronto_0220","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJohnToronto_0220"},{"LocationGroupId":18820,"LocationGroupName":"AppleTV","ParentId":3909,"label":"Toronto Catholic District School Board / Elementary / StJohnToronto_0220 / AppleTV"},{"LocationGroupId":4424,"LocationGroupName":"iOS","ParentId":3909,"label":"Toronto Catholic District School Board / Elementary / StJohnToronto_0220 / iOS"},{"LocationGroupId":18822,"LocationGroupName":"iOSLease","ParentId":3909,"label":"Toronto Catholic District School Board / Elementary / StJohnToronto_0220 / iOSLease"},{"LocationGroupId":4152,"LocationGroupName":"StJohnVianney_0392","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJohnVianney_0392"},{"LocationGroupId":18823,"LocationGroupName":"AppleTV","ParentId":4152,"label":"Toronto Catholic District School Board / Elementary / StJohnVianney_0392 / AppleTV"},{"LocationGroupId":4425,"LocationGroupName":"iOS","ParentId":4152,"label":"Toronto Catholic District School Board / Elementary / StJohnVianney_0392 / iOS"},{"LocationGroupId":18824,"LocationGroupName":"iOSLease","ParentId":4152,"label":"Toronto Catholic District School Board / Elementary / StJohnVianney_0392 / iOSLease"},{"LocationGroupId":3975,"LocationGroupName":"StJosaphat_0291","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJosaphat_0291"},{"LocationGroupId":18825,"LocationGroupName":"AppleTV","ParentId":3975,"label":"Toronto Catholic District School Board / Elementary / StJosaphat_0291 / AppleTV"},{"LocationGroupId":4426,"LocationGroupName":"iOS","ParentId":3975,"label":"Toronto Catholic District School Board / Elementary / StJosaphat_0291 / iOS"},{"LocationGroupId":18826,"LocationGroupName":"iOSLease","ParentId":3975,"label":"Toronto Catholic District School Board / Elementary / StJosaphat_0291 / iOSLease"},{"LocationGroupId":3898,"LocationGroupName":"StJoseph_0209","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJoseph_0209"},{"LocationGroupId":18827,"LocationGroupName":"AppleTV","ParentId":3898,"label":"Toronto Catholic District School Board / Elementary / StJoseph_0209 / AppleTV"},{"LocationGroupId":4427,"LocationGroupName":"iOS","ParentId":3898,"label":"Toronto Catholic District School Board / Elementary / StJoseph_0209 / iOS"},{"LocationGroupId":18828,"LocationGroupName":"iOSLease","ParentId":3898,"label":"Toronto Catholic District School Board / Elementary / StJoseph_0209 / iOSLease"},{"LocationGroupId":3964,"LocationGroupName":"StJude_0279","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StJude_0279"},{"LocationGroupId":18829,"LocationGroupName":"AppleTV","ParentId":3964,"label":"Toronto Catholic District School Board / Elementary / StJude_0279 / AppleTV"},{"LocationGroupId":4428,"LocationGroupName":"iOS","ParentId":3964,"label":"Toronto Catholic District School Board / Elementary / StJude_0279 / iOS"},{"LocationGroupId":18830,"LocationGroupName":"iOSLease","ParentId":3964,"label":"Toronto Catholic District School Board / Elementary / StJude_0279 / iOSLease"},{"LocationGroupId":4153,"LocationGroupName":"StKateriTekakwitha_0393","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StKateriTekakwitha_0393"},{"LocationGroupId":18831,"LocationGroupName":"AppleTV","ParentId":4153,"label":"Toronto Catholic District School Board / Elementary / StKateriTekakwitha_0393 / AppleTV"},{"LocationGroupId":4157,"LocationGroupName":"iOS","ParentId":4153,"label":"Toronto Catholic District School Board / Elementary / StKateriTekakwitha_0393 / iOS"},{"LocationGroupId":18832,"LocationGroupName":"iOSLease","ParentId":4153,"label":"Toronto Catholic District School Board / Elementary / StKateriTekakwitha_0393 / iOSLease"},{"LocationGroupId":3985,"LocationGroupName":"StKevin_0300","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StKevin_0300"},{"LocationGroupId":18833,"LocationGroupName":"AppleTV","ParentId":3985,"label":"Toronto Catholic District School Board / Elementary / StKevin_0300 / AppleTV"},{"LocationGroupId":4429,"LocationGroupName":"iOS","ParentId":3985,"label":"Toronto Catholic District School Board / Elementary / StKevin_0300 / iOS"},{"LocationGroupId":18834,"LocationGroupName":"iOSLease","ParentId":3985,"label":"Toronto Catholic District School Board / Elementary / StKevin_0300 / iOSLease"},{"LocationGroupId":3949,"LocationGroupName":"StLawrence_0264","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StLawrence_0264"},{"LocationGroupId":18835,"LocationGroupName":"AppleTV","ParentId":3949,"label":"Toronto Catholic District School Board / Elementary / StLawrence_0264 / AppleTV"},{"LocationGroupId":4430,"LocationGroupName":"iOS","ParentId":3949,"label":"Toronto Catholic District School Board / Elementary / StLawrence_0264 / iOS"},{"LocationGroupId":18836,"LocationGroupName":"iOSLease","ParentId":3949,"label":"Toronto Catholic District School Board / Elementary / StLawrence_0264 / iOSLease"},{"LocationGroupId":4015,"LocationGroupName":"StLeo_0333","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StLeo_0333"},{"LocationGroupId":18837,"LocationGroupName":"AppleTV","ParentId":4015,"label":"Toronto Catholic District School Board / Elementary / StLeo_0333 / AppleTV"},{"LocationGroupId":4431,"LocationGroupName":"iOS","ParentId":4015,"label":"Toronto Catholic District School Board / Elementary / StLeo_0333 / iOS"},{"LocationGroupId":18838,"LocationGroupName":"iOSLease","ParentId":4015,"label":"Toronto Catholic District School Board / Elementary / StLeo_0333 / iOSLease"},{"LocationGroupId":4013,"LocationGroupName":"StLouis_0331","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StLouis_0331"},{"LocationGroupId":18839,"LocationGroupName":"AppleTV","ParentId":4013,"label":"Toronto Catholic District School Board / Elementary / StLouis_0331 / AppleTV"},{"LocationGroupId":4432,"LocationGroupName":"iOS","ParentId":4013,"label":"Toronto Catholic District School Board / Elementary / StLouis_0331 / iOS"},{"LocationGroupId":18840,"LocationGroupName":"iOSLease","ParentId":4013,"label":"Toronto Catholic District School Board / Elementary / StLouis_0331 / iOSLease"},{"LocationGroupId":4069,"LocationGroupName":"StLuigi_0370","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StLuigi_0370"},{"LocationGroupId":18841,"LocationGroupName":"AppleTV","ParentId":4069,"label":"Toronto Catholic District School Board / Elementary / StLuigi_0370 / AppleTV"},{"LocationGroupId":4433,"LocationGroupName":"iOS","ParentId":4069,"label":"Toronto Catholic District School Board / Elementary / StLuigi_0370 / iOS"},{"LocationGroupId":18842,"LocationGroupName":"iOSLease","ParentId":4069,"label":"Toronto Catholic District School Board / Elementary / StLuigi_0370 / iOSLease"},{"LocationGroupId":4018,"LocationGroupName":"StMalachy_0336","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMalachy_0336"},{"LocationGroupId":18843,"LocationGroupName":"AppleTV","ParentId":4018,"label":"Toronto Catholic District School Board / Elementary / StMalachy_0336 / AppleTV"},{"LocationGroupId":4435,"LocationGroupName":"iOS","ParentId":4018,"label":"Toronto Catholic District School Board / Elementary / StMalachy_0336 / iOS"},{"LocationGroupId":18844,"LocationGroupName":"iOSLease","ParentId":4018,"label":"Toronto Catholic District School Board / Elementary / StMalachy_0336 / iOSLease"},{"LocationGroupId":3996,"LocationGroupName":"StMarcellus_0311","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMarcellus_0311"},{"LocationGroupId":18845,"LocationGroupName":"AppleTV","ParentId":3996,"label":"Toronto Catholic District School Board / Elementary / StMarcellus_0311 / AppleTV"},{"LocationGroupId":4436,"LocationGroupName":"iOS","ParentId":3996,"label":"Toronto Catholic District School Board / Elementary / StMarcellus_0311 / iOS"},{"LocationGroupId":18846,"LocationGroupName":"iOSLease","ParentId":3996,"label":"Toronto Catholic District School Board / Elementary / StMarcellus_0311 / iOSLease"},{"LocationGroupId":3929,"LocationGroupName":"StMargaret_0249","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMargaret_0249"},{"LocationGroupId":18847,"LocationGroupName":"AppleTV","ParentId":3929,"label":"Toronto Catholic District School Board / Elementary / StMargaret_0249 / AppleTV"},{"LocationGroupId":4437,"LocationGroupName":"iOS","ParentId":3929,"label":"Toronto Catholic District School Board / Elementary / StMargaret_0249 / iOS"},{"LocationGroupId":18848,"LocationGroupName":"iOSLease","ParentId":3929,"label":"Toronto Catholic District School Board / Elementary / StMargaret_0249 / iOSLease"},{"LocationGroupId":4079,"LocationGroupName":"StMargueriteBourgeoys_0381","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMargueriteBourgeoys_0381"},{"LocationGroupId":18849,"LocationGroupName":"AppleTV","ParentId":4079,"label":"Toronto Catholic District School Board / Elementary / StMargueriteBourgeoys_0381 / AppleTV"},{"LocationGroupId":4438,"LocationGroupName":"iOS","ParentId":4079,"label":"Toronto Catholic District School Board / Elementary / StMargueriteBourgeoys_0381 / iOS"},{"LocationGroupId":18850,"LocationGroupName":"iOSLease","ParentId":4079,"label":"Toronto Catholic District School Board / Elementary / StMargueriteBourgeoys_0381 / iOSLease"},{"LocationGroupId":3945,"LocationGroupName":"StMariaGoretti_0260","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMariaGoretti_0260"},{"LocationGroupId":18851,"LocationGroupName":"AppleTV","ParentId":3945,"label":"Toronto Catholic District School Board / Elementary / StMariaGoretti_0260 / AppleTV"},{"LocationGroupId":4439,"LocationGroupName":"iOS","ParentId":3945,"label":"Toronto Catholic District School Board / Elementary / StMariaGoretti_0260 / iOS"},{"LocationGroupId":18852,"LocationGroupName":"iOSLease","ParentId":3945,"label":"Toronto Catholic District School Board / Elementary / StMariaGoretti_0260 / iOSLease"},{"LocationGroupId":4014,"LocationGroupName":"StMark_0332","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMark_0332"},{"LocationGroupId":18853,"LocationGroupName":"AppleTV","ParentId":4014,"label":"Toronto Catholic District School Board / Elementary / StMark_0332 / AppleTV"},{"LocationGroupId":4440,"LocationGroupName":"iOS","ParentId":4014,"label":"Toronto Catholic District School Board / Elementary / StMark_0332 / iOS"},{"LocationGroupId":18854,"LocationGroupName":"iOSLease","ParentId":4014,"label":"Toronto Catholic District School Board / Elementary / StMark_0332 / iOSLease"},{"LocationGroupId":4025,"LocationGroupName":"StMartha_0344","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMartha_0344"},{"LocationGroupId":18855,"LocationGroupName":"AppleTV","ParentId":4025,"label":"Toronto Catholic District School Board / Elementary / StMartha_0344 / AppleTV"},{"LocationGroupId":4441,"LocationGroupName":"iOS","ParentId":4025,"label":"Toronto Catholic District School Board / Elementary / StMartha_0344 / iOS"},{"LocationGroupId":18856,"LocationGroupName":"iOSLease","ParentId":4025,"label":"Toronto Catholic District School Board / Elementary / StMartha_0344 / iOSLease"},{"LocationGroupId":3971,"LocationGroupName":"StMartindePorres_0286","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMartindePorres_0286"},{"LocationGroupId":18857,"LocationGroupName":"AppleTV","ParentId":3971,"label":"Toronto Catholic District School Board / Elementary / StMartindePorres_0286 / AppleTV"},{"LocationGroupId":4442,"LocationGroupName":"iOS","ParentId":3971,"label":"Toronto Catholic District School Board / Elementary / StMartindePorres_0286 / iOS"},{"LocationGroupId":18858,"LocationGroupName":"iOSLease","ParentId":3971,"label":"Toronto Catholic District School Board / Elementary / StMartindePorres_0286 / iOSLease"},{"LocationGroupId":3181,"LocationGroupName":"StMary_0205","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMary_0205"},{"LocationGroupId":18859,"LocationGroupName":"AppleTV","ParentId":3181,"label":"Toronto Catholic District School Board / Elementary / StMary_0205 / AppleTV"},{"LocationGroupId":3617,"LocationGroupName":"Devices","ParentId":3181,"label":"Toronto Catholic District School Board / Elementary / StMary_0205 / Devices"},{"LocationGroupId":4443,"LocationGroupName":"iOS","ParentId":3181,"label":"Toronto Catholic District School Board / Elementary / StMary_0205 / iOS"},{"LocationGroupId":18860,"LocationGroupName":"iOSLease","ParentId":3181,"label":"Toronto Catholic District School Board / Elementary / StMary_0205 / iOSLease"},{"LocationGroupId":3916,"LocationGroupName":"StMaryofTheAngels_0228","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMaryofTheAngels_0228"},{"LocationGroupId":18861,"LocationGroupName":"AppleTV","ParentId":3916,"label":"Toronto Catholic District School Board / Elementary / StMaryofTheAngels_0228 / AppleTV"},{"LocationGroupId":4444,"LocationGroupName":"iOS","ParentId":3916,"label":"Toronto Catholic District School Board / Elementary / StMaryofTheAngels_0228 / iOS"},{"LocationGroupId":18862,"LocationGroupName":"iOSLease","ParentId":3916,"label":"Toronto Catholic District School Board / Elementary / StMaryofTheAngels_0228 / iOSLease"},{"LocationGroupId":3921,"LocationGroupName":"StMatthew_0240","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMatthew_0240"},{"LocationGroupId":18863,"LocationGroupName":"AppleTV","ParentId":3921,"label":"Toronto Catholic District School Board / Elementary / StMatthew_0240 / AppleTV"},{"LocationGroupId":4445,"LocationGroupName":"iOS","ParentId":3921,"label":"Toronto Catholic District School Board / Elementary / StMatthew_0240 / iOS"},{"LocationGroupId":18864,"LocationGroupName":"iOSLease","ParentId":3921,"label":"Toronto Catholic District School Board / Elementary / StMatthew_0240 / iOSLease"},{"LocationGroupId":4058,"LocationGroupName":"StMatthias_0357","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMatthias_0357"},{"LocationGroupId":18865,"LocationGroupName":"AppleTV","ParentId":4058,"label":"Toronto Catholic District School Board / Elementary / StMatthias_0357 / AppleTV"},{"LocationGroupId":4446,"LocationGroupName":"iOS","ParentId":4058,"label":"Toronto Catholic District School Board / Elementary / StMatthias_0357 / iOS"},{"LocationGroupId":18866,"LocationGroupName":"iOSLease","ParentId":4058,"label":"Toronto Catholic District School Board / Elementary / StMatthias_0357 / iOSLease"},{"LocationGroupId":4077,"LocationGroupName":"StMaurice_0379","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMaurice_0379"},{"LocationGroupId":18867,"LocationGroupName":"AppleTV","ParentId":4077,"label":"Toronto Catholic District School Board / Elementary / StMaurice_0379 / AppleTV"},{"LocationGroupId":4447,"LocationGroupName":"iOS","ParentId":4077,"label":"Toronto Catholic District School Board / Elementary / StMaurice_0379 / iOS"},{"LocationGroupId":18868,"LocationGroupName":"iOSLease","ParentId":4077,"label":"Toronto Catholic District School Board / Elementary / StMaurice_0379 / iOSLease"},{"LocationGroupId":4211,"LocationGroupName":"StMaximilianKolbe_0412","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMaximilianKolbe_0412"},{"LocationGroupId":18869,"LocationGroupName":"AppleTV","ParentId":4211,"label":"Toronto Catholic District School Board / Elementary / StMaximilianKolbe_0412 / AppleTV"},{"LocationGroupId":4448,"LocationGroupName":"iOS","ParentId":4211,"label":"Toronto Catholic District School Board / Elementary / StMaximilianKolbe_0412 / iOS"},{"LocationGroupId":18870,"LocationGroupName":"iOSLease","ParentId":4211,"label":"Toronto Catholic District School Board / Elementary / StMaximilianKolbe_0412 / iOSLease"},{"LocationGroupId":4150,"LocationGroupName":"StMichael_0387","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMichael_0387"},{"LocationGroupId":18871,"LocationGroupName":"AppleTV","ParentId":4150,"label":"Toronto Catholic District School Board / Elementary / StMichael_0387 / AppleTV"},{"LocationGroupId":4449,"LocationGroupName":"iOS","ParentId":4150,"label":"Toronto Catholic District School Board / Elementary / StMichael_0387 / iOS"},{"LocationGroupId":18872,"LocationGroupName":"iOSLease","ParentId":4150,"label":"Toronto Catholic District School Board / Elementary / StMichael_0387 / iOSLease"},{"LocationGroupId":3904,"LocationGroupName":"StMichaelChoirJr_0217","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMichaelChoirJr_0217"},{"LocationGroupId":18873,"LocationGroupName":"AppleTV","ParentId":3904,"label":"Toronto Catholic District School Board / Elementary / StMichaelChoirJr_0217 / AppleTV"},{"LocationGroupId":4450,"LocationGroupName":"iOS","ParentId":3904,"label":"Toronto Catholic District School Board / Elementary / StMichaelChoirJr_0217 / iOS"},{"LocationGroupId":18874,"LocationGroupName":"iOSLease","ParentId":3904,"label":"Toronto Catholic District School Board / Elementary / StMichaelChoirJr_0217 / iOSLease"},{"LocationGroupId":3911,"LocationGroupName":"StMonica_0224","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StMonica_0224"},{"LocationGroupId":18875,"LocationGroupName":"AppleTV","ParentId":3911,"label":"Toronto Catholic District School Board / Elementary / StMonica_0224 / AppleTV"},{"LocationGroupId":4451,"LocationGroupName":"iOS","ParentId":3911,"label":"Toronto Catholic District School Board / Elementary / StMonica_0224 / iOS"},{"LocationGroupId":18876,"LocationGroupName":"iOSLease","ParentId":3911,"label":"Toronto Catholic District School Board / Elementary / StMonica_0224 / iOSLease"},{"LocationGroupId":3993,"LocationGroupName":"StNicholas_0308","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StNicholas_0308"},{"LocationGroupId":18877,"LocationGroupName":"AppleTV","ParentId":3993,"label":"Toronto Catholic District School Board / Elementary / StNicholas_0308 / AppleTV"},{"LocationGroupId":4452,"LocationGroupName":"iOS","ParentId":3993,"label":"Toronto Catholic District School Board / Elementary / StNicholas_0308 / iOS"},{"LocationGroupId":18878,"LocationGroupName":"iOSLease","ParentId":3993,"label":"Toronto Catholic District School Board / Elementary / StNicholas_0308 / iOSLease"},{"LocationGroupId":4056,"LocationGroupName":"StNicholasofBari_0355","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StNicholasofBari_0355"},{"LocationGroupId":18879,"LocationGroupName":"AppleTV","ParentId":4056,"label":"Toronto Catholic District School Board / Elementary / StNicholasofBari_0355 / AppleTV"},{"LocationGroupId":4453,"LocationGroupName":"iOS","ParentId":4056,"label":"Toronto Catholic District School Board / Elementary / StNicholasofBari_0355 / iOS"},{"LocationGroupId":18880,"LocationGroupName":"iOSLease","ParentId":4056,"label":"Toronto Catholic District School Board / Elementary / StNicholasofBari_0355 / iOSLease"},{"LocationGroupId":3994,"LocationGroupName":"StNorbert_0309","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StNorbert_0309"},{"LocationGroupId":18881,"LocationGroupName":"AppleTV","ParentId":3994,"label":"Toronto Catholic District School Board / Elementary / StNorbert_0309 / AppleTV"},{"LocationGroupId":4454,"LocationGroupName":"iOS","ParentId":3994,"label":"Toronto Catholic District School Board / Elementary / StNorbert_0309 / iOS"},{"LocationGroupId":18882,"LocationGroupName":"iOSLease","ParentId":3994,"label":"Toronto Catholic District School Board / Elementary / StNorbert_0309 / iOSLease"},{"LocationGroupId":3967,"LocationGroupName":"StPaschalBaylon_0283","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StPaschalBaylon_0283"},{"LocationGroupId":18883,"LocationGroupName":"AppleTV","ParentId":3967,"label":"Toronto Catholic District School Board / Elementary / StPaschalBaylon_0283 / AppleTV"},{"LocationGroupId":4455,"LocationGroupName":"iOS","ParentId":3967,"label":"Toronto Catholic District School Board / Elementary / StPaschalBaylon_0283 / iOS"},{"LocationGroupId":18884,"LocationGroupName":"iOSLease","ParentId":3967,"label":"Toronto Catholic District School Board / Elementary / StPaschalBaylon_0283 / iOSLease"},{"LocationGroupId":2726,"LocationGroupName":"StPaul_0204","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StPaul_0204"},{"LocationGroupId":18885,"LocationGroupName":"AppleTV","ParentId":2726,"label":"Toronto Catholic District School Board / Elementary / StPaul_0204 / AppleTV"},{"LocationGroupId":4456,"LocationGroupName":"iOS","ParentId":2726,"label":"Toronto Catholic District School Board / Elementary / StPaul_0204 / iOS"},{"LocationGroupId":18886,"LocationGroupName":"iOSLease","ParentId":2726,"label":"Toronto Catholic District School Board / Elementary / StPaul_0204 / iOSLease"},{"LocationGroupId":3919,"LocationGroupName":"StPiusX_0237","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StPiusX_0237"},{"LocationGroupId":18898,"LocationGroupName":"AppleTV","ParentId":3919,"label":"Toronto Catholic District School Board / Elementary / StPiusX_0237 / AppleTV"},{"LocationGroupId":4458,"LocationGroupName":"iOS","ParentId":3919,"label":"Toronto Catholic District School Board / Elementary / StPiusX_0237 / iOS"},{"LocationGroupId":18899,"LocationGroupName":"iOSLease","ParentId":3919,"label":"Toronto Catholic District School Board / Elementary / StPiusX_0237 / iOSLease"},{"LocationGroupId":3970,"LocationGroupName":"StRaphael_0285","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StRaphael_0285"},{"LocationGroupId":18900,"LocationGroupName":"AppleTV","ParentId":3970,"label":"Toronto Catholic District School Board / Elementary / StRaphael_0285 / AppleTV"},{"LocationGroupId":4459,"LocationGroupName":"iOS","ParentId":3970,"label":"Toronto Catholic District School Board / Elementary / StRaphael_0285 / iOS"},{"LocationGroupId":18901,"LocationGroupName":"iOSLease","ParentId":3970,"label":"Toronto Catholic District School Board / Elementary / StRaphael_0285 / iOSLease"},{"LocationGroupId":3977,"LocationGroupName":"StRaymond_0293","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StRaymond_0293"},{"LocationGroupId":18902,"LocationGroupName":"AppleTV","ParentId":3977,"label":"Toronto Catholic District School Board / Elementary / StRaymond_0293 / AppleTV"},{"LocationGroupId":4460,"LocationGroupName":"iOS","ParentId":3977,"label":"Toronto Catholic District School Board / Elementary / StRaymond_0293 / iOS"},{"LocationGroupId":18903,"LocationGroupName":"iOSLease","ParentId":3977,"label":"Toronto Catholic District School Board / Elementary / StRaymond_0293 / iOSLease"},{"LocationGroupId":4197,"LocationGroupName":"StReneGoupil_0408","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StReneGoupil_0408"},{"LocationGroupId":18904,"LocationGroupName":"AppleTV","ParentId":4197,"label":"Toronto Catholic District School Board / Elementary / StReneGoupil_0408 / AppleTV"},{"LocationGroupId":4461,"LocationGroupName":"iOS","ParentId":4197,"label":"Toronto Catholic District School Board / Elementary / StReneGoupil_0408 / iOS"},{"LocationGroupId":18905,"LocationGroupName":"iOSLease","ParentId":4197,"label":"Toronto Catholic District School Board / Elementary / StReneGoupil_0408 / iOSLease"},{"LocationGroupId":3986,"LocationGroupName":"StRichard_0301","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StRichard_0301"},{"LocationGroupId":18906,"LocationGroupName":"AppleTV","ParentId":3986,"label":"Toronto Catholic District School Board / Elementary / StRichard_0301 / AppleTV"},{"LocationGroupId":4462,"LocationGroupName":"iOS","ParentId":3986,"label":"Toronto Catholic District School Board / Elementary / StRichard_0301 / iOS"},{"LocationGroupId":18907,"LocationGroupName":"iOSLease","ParentId":3986,"label":"Toronto Catholic District School Board / Elementary / StRichard_0301 / iOSLease"},{"LocationGroupId":3907,"LocationGroupName":"StRita_0216","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StRita_0216"},{"LocationGroupId":18908,"LocationGroupName":"AppleTV","ParentId":3907,"label":"Toronto Catholic District School Board / Elementary / StRita_0216 / AppleTV"},{"LocationGroupId":4463,"LocationGroupName":"iOS","ParentId":3907,"label":"Toronto Catholic District School Board / Elementary / StRita_0216 / iOS"},{"LocationGroupId":18909,"LocationGroupName":"iOSLease","ParentId":3907,"label":"Toronto Catholic District School Board / Elementary / StRita_0216 / iOSLease"},{"LocationGroupId":3982,"LocationGroupName":"StRobert_0297","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StRobert_0297"},{"LocationGroupId":18910,"LocationGroupName":"AppleTV","ParentId":3982,"label":"Toronto Catholic District School Board / Elementary / StRobert_0297 / AppleTV"},{"LocationGroupId":4464,"LocationGroupName":"iOS","ParentId":3982,"label":"Toronto Catholic District School Board / Elementary / StRobert_0297 / iOS"},{"LocationGroupId":18911,"LocationGroupName":"iOSLease","ParentId":3982,"label":"Toronto Catholic District School Board / Elementary / StRobert_0297 / iOSLease"},{"LocationGroupId":4003,"LocationGroupName":"StRoch_0320","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StRoch_0320"},{"LocationGroupId":18912,"LocationGroupName":"AppleTV","ParentId":4003,"label":"Toronto Catholic District School Board / Elementary / StRoch_0320 / AppleTV"},{"LocationGroupId":4465,"LocationGroupName":"iOS","ParentId":4003,"label":"Toronto Catholic District School Board / Elementary / StRoch_0320 / iOS"},{"LocationGroupId":18913,"LocationGroupName":"iOSLease","ParentId":4003,"label":"Toronto Catholic District School Board / Elementary / StRoch_0320 / iOSLease"},{"LocationGroupId":3953,"LocationGroupName":"StRoseOfLima_0269","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StRoseOfLima_0269"},{"LocationGroupId":18914,"LocationGroupName":"AppleTV","ParentId":3953,"label":"Toronto Catholic District School Board / Elementary / StRoseOfLima_0269 / AppleTV"},{"LocationGroupId":4466,"LocationGroupName":"iOS","ParentId":3953,"label":"Toronto Catholic District School Board / Elementary / StRoseOfLima_0269 / iOS"},{"LocationGroupId":18915,"LocationGroupName":"iOSLease","ParentId":3953,"label":"Toronto Catholic District School Board / Elementary / StRoseOfLima_0269 / iOSLease"},{"LocationGroupId":4033,"LocationGroupName":"StSebastian_0354","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StSebastian_0354"},{"LocationGroupId":18916,"LocationGroupName":"AppleTV","ParentId":4033,"label":"Toronto Catholic District School Board / Elementary / StSebastian_0354 / AppleTV"},{"LocationGroupId":4467,"LocationGroupName":"iOS","ParentId":4033,"label":"Toronto Catholic District School Board / Elementary / StSebastian_0354 / iOS"},{"LocationGroupId":18917,"LocationGroupName":"iOSLease","ParentId":4033,"label":"Toronto Catholic District School Board / Elementary / StSebastian_0354 / iOSLease"},{"LocationGroupId":4138,"LocationGroupName":"StSimon_0383","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StSimon_0383"},{"LocationGroupId":18918,"LocationGroupName":"AppleTV","ParentId":4138,"label":"Toronto Catholic District School Board / Elementary / StSimon_0383 / AppleTV"},{"LocationGroupId":4468,"LocationGroupName":"iOS","ParentId":4138,"label":"Toronto Catholic District School Board / Elementary / StSimon_0383 / iOS"},{"LocationGroupId":18919,"LocationGroupName":"iOSLease","ParentId":4138,"label":"Toronto Catholic District School Board / Elementary / StSimon_0383 / iOSLease"},{"LocationGroupId":3969,"LocationGroupName":"StStephen_0284","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StStephen_0284"},{"LocationGroupId":18920,"LocationGroupName":"AppleTV","ParentId":3969,"label":"Toronto Catholic District School Board / Elementary / StStephen_0284 / AppleTV"},{"LocationGroupId":4469,"LocationGroupName":"iOS","ParentId":3969,"label":"Toronto Catholic District School Board / Elementary / StStephen_0284 / iOS"},{"LocationGroupId":18921,"LocationGroupName":"iOSLease","ParentId":3969,"label":"Toronto Catholic District School Board / Elementary / StStephen_0284 / iOSLease"},{"LocationGroupId":4072,"LocationGroupName":"StSylvester_0373","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StSylvester_0373"},{"LocationGroupId":18922,"LocationGroupName":"AppleTV","ParentId":4072,"label":"Toronto Catholic District School Board / Elementary / StSylvester_0373 / AppleTV"},{"LocationGroupId":4470,"LocationGroupName":"iOS","ParentId":4072,"label":"Toronto Catholic District School Board / Elementary / StSylvester_0373 / iOS"},{"LocationGroupId":18923,"LocationGroupName":"iOSLease","ParentId":4072,"label":"Toronto Catholic District School Board / Elementary / StSylvester_0373 / iOSLease"},{"LocationGroupId":3951,"LocationGroupName":"StTeresa_0266","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StTeresa_0266"},{"LocationGroupId":18924,"LocationGroupName":"AppleTV","ParentId":3951,"label":"Toronto Catholic District School Board / Elementary / StTeresa_0266 / AppleTV"},{"LocationGroupId":4471,"LocationGroupName":"iOS","ParentId":3951,"label":"Toronto Catholic District School Board / Elementary / StTeresa_0266 / iOS"},{"LocationGroupId":18925,"LocationGroupName":"iOSLease","ParentId":3951,"label":"Toronto Catholic District School Board / Elementary / StTeresa_0266 / iOSLease"},{"LocationGroupId":3928,"LocationGroupName":"StTheresaShrine_0248","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StTheresaShrine_0248"},{"LocationGroupId":18926,"LocationGroupName":"AppleTV","ParentId":3928,"label":"Toronto Catholic District School Board / Elementary / StTheresaShrine_0248 / AppleTV"},{"LocationGroupId":4472,"LocationGroupName":"iOS","ParentId":3928,"label":"Toronto Catholic District School Board / Elementary / StTheresaShrine_0248 / iOS"},{"LocationGroupId":18927,"LocationGroupName":"iOSLease","ParentId":3928,"label":"Toronto Catholic District School Board / Elementary / StTheresaShrine_0248 / iOSLease"},{"LocationGroupId":3918,"LocationGroupName":"StThomasAquinas_0236","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StThomasAquinas_0236"},{"LocationGroupId":18928,"LocationGroupName":"AppleTV","ParentId":3918,"label":"Toronto Catholic District School Board / Elementary / StThomasAquinas_0236 / AppleTV"},{"LocationGroupId":4474,"LocationGroupName":"iOS","ParentId":3918,"label":"Toronto Catholic District School Board / Elementary / StThomasAquinas_0236 / iOS"},{"LocationGroupId":18929,"LocationGroupName":"iOSLease","ParentId":3918,"label":"Toronto Catholic District School Board / Elementary / StThomasAquinas_0236 / iOSLease"},{"LocationGroupId":4004,"LocationGroupName":"StThomasMore_0322","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StThomasMore_0322"},{"LocationGroupId":18930,"LocationGroupName":"AppleTV","ParentId":4004,"label":"Toronto Catholic District School Board / Elementary / StThomasMore_0322 / AppleTV"},{"LocationGroupId":4475,"LocationGroupName":"iOS","ParentId":4004,"label":"Toronto Catholic District School Board / Elementary / StThomasMore_0322 / iOS"},{"LocationGroupId":18931,"LocationGroupName":"iOSLease","ParentId":4004,"label":"Toronto Catholic District School Board / Elementary / StThomasMore_0322 / iOSLease"},{"LocationGroupId":3983,"LocationGroupName":"StTimothy_0298","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StTimothy_0298"},{"LocationGroupId":18932,"LocationGroupName":"AppleTV","ParentId":3983,"label":"Toronto Catholic District School Board / Elementary / StTimothy_0298 / AppleTV"},{"LocationGroupId":4476,"LocationGroupName":"iOS","ParentId":3983,"label":"Toronto Catholic District School Board / Elementary / StTimothy_0298 / iOS"},{"LocationGroupId":18933,"LocationGroupName":"iOSLease","ParentId":3983,"label":"Toronto Catholic District School Board / Elementary / StTimothy_0298 / iOSLease"},{"LocationGroupId":3991,"LocationGroupName":"StUrsula_0306","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StUrsula_0306"},{"LocationGroupId":18934,"LocationGroupName":"AppleTV","ParentId":3991,"label":"Toronto Catholic District School Board / Elementary / StUrsula_0306 / AppleTV"},{"LocationGroupId":4477,"LocationGroupName":"iOS","ParentId":3991,"label":"Toronto Catholic District School Board / Elementary / StUrsula_0306 / iOS"},{"LocationGroupId":18935,"LocationGroupName":"iOSLease","ParentId":3991,"label":"Toronto Catholic District School Board / Elementary / StUrsula_0306 / iOSLease"},{"LocationGroupId":4020,"LocationGroupName":"StVictor_0338","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StVictor_0338"},{"LocationGroupId":18936,"LocationGroupName":"AppleTV","ParentId":4020,"label":"Toronto Catholic District School Board / Elementary / StVictor_0338 / AppleTV"},{"LocationGroupId":4478,"LocationGroupName":"iOS","ParentId":4020,"label":"Toronto Catholic District School Board / Elementary / StVictor_0338 / iOS"},{"LocationGroupId":18937,"LocationGroupName":"iOSLease","ParentId":4020,"label":"Toronto Catholic District School Board / Elementary / StVictor_0338 / iOSLease"},{"LocationGroupId":3915,"LocationGroupName":"StVincentdePaul_0227","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StVincentdePaul_0227"},{"LocationGroupId":18938,"LocationGroupName":"AppleTV","ParentId":3915,"label":"Toronto Catholic District School Board / Elementary / StVincentdePaul_0227 / AppleTV"},{"LocationGroupId":4479,"LocationGroupName":"iOS","ParentId":3915,"label":"Toronto Catholic District School Board / Elementary / StVincentdePaul_0227 / iOS"},{"LocationGroupId":18939,"LocationGroupName":"iOSLease","ParentId":3915,"label":"Toronto Catholic District School Board / Elementary / StVincentdePaul_0227 / iOSLease"},{"LocationGroupId":3997,"LocationGroupName":"StWilfrid_0313","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StWilfrid_0313"},{"LocationGroupId":18896,"LocationGroupName":"AppleTV","ParentId":3997,"label":"Toronto Catholic District School Board / Elementary / StWilfrid_0313 / AppleTV"},{"LocationGroupId":4480,"LocationGroupName":"iOS","ParentId":3997,"label":"Toronto Catholic District School Board / Elementary / StWilfrid_0313 / iOS"},{"LocationGroupId":18897,"LocationGroupName":"iOSLease","ParentId":3997,"label":"Toronto Catholic District School Board / Elementary / StWilfrid_0313 / iOSLease"},{"LocationGroupId":3988,"LocationGroupName":"StWilliam_0303","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / StWilliam_0303"},{"LocationGroupId":18894,"LocationGroupName":"AppleTV","ParentId":3988,"label":"Toronto Catholic District School Board / Elementary / StWilliam_0303 / AppleTV"},{"LocationGroupId":4481,"LocationGroupName":"iOS","ParentId":3988,"label":"Toronto Catholic District School Board / Elementary / StWilliam_0303 / iOS"},{"LocationGroupId":18895,"LocationGroupName":"iOSLease","ParentId":3988,"label":"Toronto Catholic District School Board / Elementary / StWilliam_0303 / iOSLease"},{"LocationGroupId":4213,"LocationGroupName":"TheDivineInfant_0414","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / TheDivineInfant_0414"},{"LocationGroupId":18893,"LocationGroupName":"AppleTV","ParentId":4213,"label":"Toronto Catholic District School Board / Elementary / TheDivineInfant_0414 / AppleTV"},{"LocationGroupId":4482,"LocationGroupName":"iOS","ParentId":4213,"label":"Toronto Catholic District School Board / Elementary / TheDivineInfant_0414 / iOS"},{"LocationGroupId":17965,"LocationGroupName":"iOSLease","ParentId":4213,"label":"Toronto Catholic District School Board / Elementary / TheDivineInfant_0414 / iOSLease"},{"LocationGroupId":17761,"LocationGroupName":"TheHolyTrinity_0417","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / TheHolyTrinity_0417"},{"LocationGroupId":18892,"LocationGroupName":"AppleTV","ParentId":17761,"label":"Toronto Catholic District School Board / Elementary / TheHolyTrinity_0417 / AppleTV"},{"LocationGroupId":17762,"LocationGroupName":"iOS","ParentId":17761,"label":"Toronto Catholic District School Board / Elementary / TheHolyTrinity_0417 / iOS"},{"LocationGroupId":17964,"LocationGroupName":"iOSLease","ParentId":17761,"label":"Toronto Catholic District School Board / Elementary / TheHolyTrinity_0417 / iOSLease"},{"LocationGroupId":4220,"LocationGroupName":"TrainingSite_0982","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / TrainingSite_0982"},{"LocationGroupId":18890,"LocationGroupName":"AppleTV","ParentId":4220,"label":"Toronto Catholic District School Board / Elementary / TrainingSite_0982 / AppleTV"},{"LocationGroupId":4483,"LocationGroupName":"iOS","ParentId":4220,"label":"Toronto Catholic District School Board / Elementary / TrainingSite_0982 / iOS"},{"LocationGroupId":18891,"LocationGroupName":"iOSLease","ParentId":4220,"label":"Toronto Catholic District School Board / Elementary / TrainingSite_0982 / iOSLease"},{"LocationGroupId":3960,"LocationGroupName":"Transfiguration_0276","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / Transfiguration_0276"},{"LocationGroupId":18889,"LocationGroupName":"AppleTV","ParentId":3960,"label":"Toronto Catholic District School Board / Elementary / Transfiguration_0276 / AppleTV"},{"LocationGroupId":4484,"LocationGroupName":"iOS","ParentId":3960,"label":"Toronto Catholic District School Board / Elementary / Transfiguration_0276 / iOS"},{"LocationGroupId":17963,"LocationGroupName":"iOSLease","ParentId":3960,"label":"Toronto Catholic District School Board / Elementary / Transfiguration_0276 / iOSLease"},{"LocationGroupId":4194,"LocationGroupName":"VenerableJohnMerlini_0397","ParentId":2717,"label":"Toronto Catholic District School Board / Elementary / VenerableJohnMerlini_0397"},{"LocationGroupId":18887,"LocationGroupName":"AppleTV","ParentId":4194,"label":"Toronto Catholic District School Board / Elementary / VenerableJohnMerlini_0397 / AppleTV"},{"LocationGroupId":4485,"LocationGroupName":"iOS","ParentId":4194,"label":"Toronto Catholic District School Board / Elementary / VenerableJohnMerlini_0397 / iOS"},{"LocationGroupId":18888,"LocationGroupName":"iOSLease","ParentId":4194,"label":"Toronto Catholic District School Board / Elementary / VenerableJohnMerlini_0397 / iOSLease"},{"LocationGroupId":2722,"LocationGroupName":"Secondary","ParentId":2673,"label":"Toronto Catholic District School Board / Secondary"},{"LocationGroupId":7123,"LocationGroupName":"ArchbishopRomero_0555","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / ArchbishopRomero_0555"},{"LocationGroupId":18940,"LocationGroupName":"AppleTV","ParentId":7123,"label":"Toronto Catholic District School Board / Secondary / ArchbishopRomero_0555 / AppleTV"},{"LocationGroupId":7147,"LocationGroupName":"iOS","ParentId":7123,"label":"Toronto Catholic District School Board / Secondary / ArchbishopRomero_0555 / iOS"},{"LocationGroupId":18941,"LocationGroupName":"iOSLease","ParentId":7123,"label":"Toronto Catholic District School Board / Secondary / ArchbishopRomero_0555 / iOSLease"},{"LocationGroupId":7120,"LocationGroupName":"BishopAllen_0549","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / BishopAllen_0549"},{"LocationGroupId":18942,"LocationGroupName":"AppleTV","ParentId":7120,"label":"Toronto Catholic District School Board / Secondary / BishopAllen_0549 / AppleTV"},{"LocationGroupId":7145,"LocationGroupName":"iOS","ParentId":7120,"label":"Toronto Catholic District School Board / Secondary / BishopAllen_0549 / iOS"},{"LocationGroupId":18943,"LocationGroupName":"iOSLease","ParentId":7120,"label":"Toronto Catholic District School Board / Secondary / BishopAllen_0549 / iOSLease"},{"LocationGroupId":7115,"LocationGroupName":"BishopMarroccoThomasMerton_0545","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / BishopMarroccoThomasMerton_0545"},{"LocationGroupId":18944,"LocationGroupName":"AppleTV","ParentId":7115,"label":"Toronto Catholic District School Board / Secondary / BishopMarroccoThomasMerton_0545 / AppleTV"},{"LocationGroupId":7143,"LocationGroupName":"iOS","ParentId":7115,"label":"Toronto Catholic District School Board / Secondary / BishopMarroccoThomasMerton_0545 / iOS"},{"LocationGroupId":18945,"LocationGroupName":"iOSLease","ParentId":7115,"label":"Toronto Catholic District School Board / Secondary / BishopMarroccoThomasMerton_0545 / iOSLease"},{"LocationGroupId":6731,"LocationGroupName":"BlessedCardinalNewman_0519","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / BlessedCardinalNewman_0519"},{"LocationGroupId":18946,"LocationGroupName":"AppleTV","ParentId":6731,"label":"Toronto Catholic District School Board / Secondary / BlessedCardinalNewman_0519 / AppleTV"},{"LocationGroupId":6735,"LocationGroupName":"iOS","ParentId":6731,"label":"Toronto Catholic District School Board / Secondary / BlessedCardinalNewman_0519 / iOS"},{"LocationGroupId":18947,"LocationGroupName":"iOSLease","ParentId":6731,"label":"Toronto Catholic District School Board / Secondary / BlessedCardinalNewman_0519 / iOSLease"},{"LocationGroupId":7110,"LocationGroupName":"BlessedMotherTeresa_0538","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / BlessedMotherTeresa_0538"},{"LocationGroupId":18948,"LocationGroupName":"AppleTV","ParentId":7110,"label":"Toronto Catholic District School Board / Secondary / BlessedMotherTeresa_0538 / AppleTV"},{"LocationGroupId":7141,"LocationGroupName":"iOS","ParentId":7110,"label":"Toronto Catholic District School Board / Secondary / BlessedMotherTeresa_0538 / iOS"},{"LocationGroupId":18949,"LocationGroupName":"iOSLease","ParentId":7110,"label":"Toronto Catholic District School Board / Secondary / BlessedMotherTeresa_0538 / iOSLease"},{"LocationGroupId":7094,"LocationGroupName":"Brebeuf_0508","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / Brebeuf_0508"},{"LocationGroupId":18950,"LocationGroupName":"AppleTV","ParentId":7094,"label":"Toronto Catholic District School Board / Secondary / Brebeuf_0508 / AppleTV"},{"LocationGroupId":7129,"LocationGroupName":"iOS","ParentId":7094,"label":"Toronto Catholic District School Board / Secondary / Brebeuf_0508 / iOS"},{"LocationGroupId":18951,"LocationGroupName":"iOSLease","ParentId":7094,"label":"Toronto Catholic District School Board / Secondary / Brebeuf_0508 / iOSLease"},{"LocationGroupId":7124,"LocationGroupName":"CardinalCarterAcademy_0556","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / CardinalCarterAcademy_0556"},{"LocationGroupId":18952,"LocationGroupName":"AppleTV","ParentId":7124,"label":"Toronto Catholic District School Board / Secondary / CardinalCarterAcademy_0556 / AppleTV"},{"LocationGroupId":7148,"LocationGroupName":"iOS","ParentId":7124,"label":"Toronto Catholic District School Board / Secondary / CardinalCarterAcademy_0556 / iOS"},{"LocationGroupId":18953,"LocationGroupName":"iOSLease","ParentId":7124,"label":"Toronto Catholic District School Board / Secondary / CardinalCarterAcademy_0556 / iOSLease"},{"LocationGroupId":7097,"LocationGroupName":"Chaminade_0509","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / Chaminade_0509"},{"LocationGroupId":16769,"LocationGroupName":"Android","ParentId":7097,"label":"Toronto Catholic District School Board / Secondary / Chaminade_0509 / Android"},{"LocationGroupId":18954,"LocationGroupName":"AppleTV","ParentId":7097,"label":"Toronto Catholic District School Board / Secondary / Chaminade_0509 / AppleTV"},{"LocationGroupId":7130,"LocationGroupName":"iOS","ParentId":7097,"label":"Toronto Catholic District School Board / Secondary / Chaminade_0509 / iOS"},{"LocationGroupId":18955,"LocationGroupName":"iOSLease","ParentId":7097,"label":"Toronto Catholic District School Board / Secondary / Chaminade_0509 / iOSLease"},{"LocationGroupId":7031,"LocationGroupName":"DanteAlighieri_0522","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / DanteAlighieri_0522"},{"LocationGroupId":18956,"LocationGroupName":"AppleTV","ParentId":7031,"label":"Toronto Catholic District School Board / Secondary / DanteAlighieri_0522 / AppleTV"},{"LocationGroupId":7032,"LocationGroupName":"iOS","ParentId":7031,"label":"Toronto Catholic District School Board / Secondary / DanteAlighieri_0522 / iOS"},{"LocationGroupId":18958,"LocationGroupName":"iOSLease","ParentId":7031,"label":"Toronto Catholic District School Board / Secondary / DanteAlighieri_0522 / iOSLease"},{"LocationGroupId":6728,"LocationGroupName":"DonBosco_0525","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / DonBosco_0525"},{"LocationGroupId":6734,"LocationGroupName":"iOS","ParentId":6728,"label":"Toronto Catholic District School Board / Secondary / DonBosco_0525 / iOS"},{"LocationGroupId":7104,"LocationGroupName":"FrancisLiberman_0524","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / FrancisLiberman_0524"},{"LocationGroupId":18959,"LocationGroupName":"AppleTV","ParentId":7104,"label":"Toronto Catholic District School Board / Secondary / FrancisLiberman_0524 / AppleTV"},{"LocationGroupId":7134,"LocationGroupName":"iOS","ParentId":7104,"label":"Toronto Catholic District School Board / Secondary / FrancisLiberman_0524 / iOS"},{"LocationGroupId":18960,"LocationGroupName":"iOSLease","ParentId":7104,"label":"Toronto Catholic District School Board / Secondary / FrancisLiberman_0524 / iOSLease"},{"LocationGroupId":7103,"LocationGroupName":"FrHenryCarr_0521","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / FrHenryCarr_0521"},{"LocationGroupId":18961,"LocationGroupName":"AppleTV","ParentId":7103,"label":"Toronto Catholic District School Board / Secondary / FrHenryCarr_0521 / AppleTV"},{"LocationGroupId":7135,"LocationGroupName":"iOS","ParentId":7103,"label":"Toronto Catholic District School Board / Secondary / FrHenryCarr_0521 / iOS"},{"LocationGroupId":18962,"LocationGroupName":"iOSLease","ParentId":7103,"label":"Toronto Catholic District School Board / Secondary / FrHenryCarr_0521 / iOSLease"},{"LocationGroupId":6740,"LocationGroupName":"FrJohnRedmond_0540","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / FrJohnRedmond_0540"},{"LocationGroupId":18963,"LocationGroupName":"AppleTV","ParentId":6740,"label":"Toronto Catholic District School Board / Secondary / FrJohnRedmond_0540 / AppleTV"},{"LocationGroupId":6741,"LocationGroupName":"iOS","ParentId":6740,"label":"Toronto Catholic District School Board / Secondary / FrJohnRedmond_0540 / iOS"},{"LocationGroupId":18964,"LocationGroupName":"iOSLease","ParentId":6740,"label":"Toronto Catholic District School Board / Secondary / FrJohnRedmond_0540 / iOSLease"},{"LocationGroupId":7106,"LocationGroupName":"JamesCardinalMcguigan_0527","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / JamesCardinalMcguigan_0527"},{"LocationGroupId":18965,"LocationGroupName":"AppleTV","ParentId":7106,"label":"Toronto Catholic District School Board / Secondary / JamesCardinalMcguigan_0527 / AppleTV"},{"LocationGroupId":7137,"LocationGroupName":"iOS","ParentId":7106,"label":"Toronto Catholic District School Board / Secondary / JamesCardinalMcguigan_0527 / iOS"},{"LocationGroupId":18966,"LocationGroupName":"iOSLease","ParentId":7106,"label":"Toronto Catholic District School Board / Secondary / JamesCardinalMcguigan_0527 / iOSLease"},{"LocationGroupId":7122,"LocationGroupName":"JeanVanier_0554","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / JeanVanier_0554"},{"LocationGroupId":18967,"LocationGroupName":"AppleTV","ParentId":7122,"label":"Toronto Catholic District School Board / Secondary / JeanVanier_0554 / AppleTV"},{"LocationGroupId":7146,"LocationGroupName":"iOS","ParentId":7122,"label":"Toronto Catholic District School Board / Secondary / JeanVanier_0554 / iOS"},{"LocationGroupId":18968,"LocationGroupName":"iOSLease","ParentId":7122,"label":"Toronto Catholic District School Board / Secondary / JeanVanier_0554 / iOSLease"},{"LocationGroupId":17926,"LocationGroupName":"Mac","ParentId":7122,"label":"Toronto Catholic District School Board / Secondary / JeanVanier_0554 / Mac"},{"LocationGroupId":7100,"LocationGroupName":"LorettoAbbey_0510","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / LorettoAbbey_0510"},{"LocationGroupId":18969,"LocationGroupName":"AppleTV","ParentId":7100,"label":"Toronto Catholic District School Board / Secondary / LorettoAbbey_0510 / AppleTV"},{"LocationGroupId":7131,"LocationGroupName":"iOS","ParentId":7100,"label":"Toronto Catholic District School Board / Secondary / LorettoAbbey_0510 / iOS"},{"LocationGroupId":18970,"LocationGroupName":"iOSLease","ParentId":7100,"label":"Toronto Catholic District School Board / Secondary / LorettoAbbey_0510 / iOSLease"},{"LocationGroupId":7101,"LocationGroupName":"LorettoCollege_0511","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / LorettoCollege_0511"},{"LocationGroupId":18971,"LocationGroupName":"AppleTV","ParentId":7101,"label":"Toronto Catholic District School Board / Secondary / LorettoCollege_0511 / AppleTV"},{"LocationGroupId":7132,"LocationGroupName":"iOS","ParentId":7101,"label":"Toronto Catholic District School Board / Secondary / LorettoCollege_0511 / iOS"},{"LocationGroupId":18972,"LocationGroupName":"iOSLease","ParentId":7101,"label":"Toronto Catholic District School Board / Secondary / LorettoCollege_0511 / iOSLease"},{"LocationGroupId":7092,"LocationGroupName":"Madonna_0506","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / Madonna_0506"},{"LocationGroupId":18973,"LocationGroupName":"AppleTV","ParentId":7092,"label":"Toronto Catholic District School Board / Secondary / Madonna_0506 / AppleTV"},{"LocationGroupId":7128,"LocationGroupName":"iOS","ParentId":7092,"label":"Toronto Catholic District School Board / Secondary / Madonna_0506 / iOS"},{"LocationGroupId":18974,"LocationGroupName":"iOSLease","ParentId":7092,"label":"Toronto Catholic District School Board / Secondary / Madonna_0506 / iOSLease"},{"LocationGroupId":7125,"LocationGroupName":"MarshallMcluhan_0561","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / MarshallMcluhan_0561"},{"LocationGroupId":18975,"LocationGroupName":"AppleTV","ParentId":7125,"label":"Toronto Catholic District School Board / Secondary / MarshallMcluhan_0561 / AppleTV"},{"LocationGroupId":7149,"LocationGroupName":"iOS","ParentId":7125,"label":"Toronto Catholic District School Board / Secondary / MarshallMcluhan_0561 / iOS"},{"LocationGroupId":18976,"LocationGroupName":"iOSLease","ParentId":7125,"label":"Toronto Catholic District School Board / Secondary / MarshallMcluhan_0561 / iOSLease"},{"LocationGroupId":7112,"LocationGroupName":"MaryWard_0544","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / MaryWard_0544"},{"LocationGroupId":18977,"LocationGroupName":"AppleTV","ParentId":7112,"label":"Toronto Catholic District School Board / Secondary / MaryWard_0544 / AppleTV"},{"LocationGroupId":7142,"LocationGroupName":"iOS","ParentId":7112,"label":"Toronto Catholic District School Board / Secondary / MaryWard_0544 / iOS"},{"LocationGroupId":18978,"LocationGroupName":"iOSLease","ParentId":7112,"label":"Toronto Catholic District School Board / Secondary / MaryWard_0544 / iOSLease"},{"LocationGroupId":6732,"LocationGroupName":"MichaelPowerStJoseph_0529","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / MichaelPowerStJoseph_0529"},{"LocationGroupId":18979,"LocationGroupName":"AppleTV","ParentId":6732,"label":"Toronto Catholic District School Board / Secondary / MichaelPowerStJoseph_0529 / AppleTV"},{"LocationGroupId":6736,"LocationGroupName":"iOS","ParentId":6732,"label":"Toronto Catholic District School Board / Secondary / MichaelPowerStJoseph_0529 / iOS"},{"LocationGroupId":18980,"LocationGroupName":"iOSLease","ParentId":6732,"label":"Toronto Catholic District School Board / Secondary / MichaelPowerStJoseph_0529 / iOSLease"},{"LocationGroupId":6733,"LocationGroupName":"MsgrFraser_0533","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / MsgrFraser_0533"},{"LocationGroupId":18981,"LocationGroupName":"AppleTV","ParentId":6733,"label":"Toronto Catholic District School Board / Secondary / MsgrFraser_0533 / AppleTV"},{"LocationGroupId":6737,"LocationGroupName":"iOS","ParentId":6733,"label":"Toronto Catholic District School Board / Secondary / MsgrFraser_0533 / iOS"},{"LocationGroupId":18982,"LocationGroupName":"iOSLease","ParentId":6733,"label":"Toronto Catholic District School Board / Secondary / MsgrFraser_0533 / iOSLease"},{"LocationGroupId":7108,"LocationGroupName":"MsgrFraserMidland_0532","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / MsgrFraserMidland_0532"},{"LocationGroupId":18983,"LocationGroupName":"AppleTV","ParentId":7108,"label":"Toronto Catholic District School Board / Secondary / MsgrFraserMidland_0532 / AppleTV"},{"LocationGroupId":7139,"LocationGroupName":"iOS","ParentId":7108,"label":"Toronto Catholic District School Board / Secondary / MsgrFraserMidland_0532 / iOS"},{"LocationGroupId":18984,"LocationGroupName":"iOSLease","ParentId":7108,"label":"Toronto Catholic District School Board / Secondary / MsgrFraserMidland_0532 / iOSLease"},{"LocationGroupId":7769,"LocationGroupName":"MsgrFraserOLMCAppleProgram_0765","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / MsgrFraserOLMCAppleProgram_0765"},{"LocationGroupId":18985,"LocationGroupName":"AppleTV","ParentId":7769,"label":"Toronto Catholic District School Board / Secondary / MsgrFraserOLMCAppleProgram_0765 / AppleTV"},{"LocationGroupId":7770,"LocationGroupName":"iOS","ParentId":7769,"label":"Toronto Catholic District School Board / Secondary / MsgrFraserOLMCAppleProgram_0765 / iOS"},{"LocationGroupId":18986,"LocationGroupName":"iOSLease","ParentId":7769,"label":"Toronto Catholic District School Board / Secondary / MsgrFraserOLMCAppleProgram_0765 / iOSLease"},{"LocationGroupId":7109,"LocationGroupName":"MsgrPercyJohnson_0535","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / MsgrPercyJohnson_0535"},{"LocationGroupId":18987,"LocationGroupName":"AppleTV","ParentId":7109,"label":"Toronto Catholic District School Board / Secondary / MsgrPercyJohnson_0535 / AppleTV"},{"LocationGroupId":7140,"LocationGroupName":"iOS","ParentId":7109,"label":"Toronto Catholic District School Board / Secondary / MsgrPercyJohnson_0535 / iOS"},{"LocationGroupId":18988,"LocationGroupName":"iOSLease","ParentId":7109,"label":"Toronto Catholic District School Board / Secondary / MsgrPercyJohnson_0535 / iOSLease"},{"LocationGroupId":7088,"LocationGroupName":"NeilMcneil_0502","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / NeilMcneil_0502"},{"LocationGroupId":18989,"LocationGroupName":"AppleTV","ParentId":7088,"label":"Toronto Catholic District School Board / Secondary / NeilMcneil_0502 / AppleTV"},{"LocationGroupId":7126,"LocationGroupName":"iOS","ParentId":7088,"label":"Toronto Catholic District School Board / Secondary / NeilMcneil_0502 / iOS"},{"LocationGroupId":18990,"LocationGroupName":"iOSLease","ParentId":7088,"label":"Toronto Catholic District School Board / Secondary / NeilMcneil_0502 / iOSLease"},{"LocationGroupId":2749,"LocationGroupName":"NotreDame_0501","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / NotreDame_0501"},{"LocationGroupId":18991,"LocationGroupName":"AppleTV","ParentId":2749,"label":"Toronto Catholic District School Board / Secondary / NotreDame_0501 / AppleTV"},{"LocationGroupId":6738,"LocationGroupName":"iOS","ParentId":2749,"label":"Toronto Catholic District School Board / Secondary / NotreDame_0501 / iOS"},{"LocationGroupId":18992,"LocationGroupName":"iOSLease","ParentId":2749,"label":"Toronto Catholic District School Board / Secondary / NotreDame_0501 / iOSLease"},{"LocationGroupId":7105,"LocationGroupName":"ReginaPacis_0526","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / ReginaPacis_0526"},{"LocationGroupId":18993,"LocationGroupName":"AppleTV","ParentId":7105,"label":"Toronto Catholic District School Board / Secondary / ReginaPacis_0526 / AppleTV"},{"LocationGroupId":7136,"LocationGroupName":"iOS","ParentId":7105,"label":"Toronto Catholic District School Board / Secondary / ReginaPacis_0526 / iOS"},{"LocationGroupId":18994,"LocationGroupName":"iOSLease","ParentId":7105,"label":"Toronto Catholic District School Board / Secondary / ReginaPacis_0526 / iOSLease"},{"LocationGroupId":7089,"LocationGroupName":"SenatorOConnor_0505","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / SenatorOConnor_0505"},{"LocationGroupId":17741,"LocationGroupName":"AppleTV","ParentId":7089,"label":"Toronto Catholic District School Board / Secondary / SenatorOConnor_0505 / AppleTV"},{"LocationGroupId":7127,"LocationGroupName":"iOS","ParentId":7089,"label":"Toronto Catholic District School Board / Secondary / SenatorOConnor_0505 / iOS"},{"LocationGroupId":18995,"LocationGroupName":"iOSLease","ParentId":7089,"label":"Toronto Catholic District School Board / Secondary / SenatorOConnor_0505 / iOSLease"},{"LocationGroupId":17740,"LocationGroupName":"Mac","ParentId":7089,"label":"Toronto Catholic District School Board / Secondary / SenatorOConnor_0505 / Mac"},{"LocationGroupId":7027,"LocationGroupName":"StBasil_0513","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / StBasil_0513"},{"LocationGroupId":18996,"LocationGroupName":"AppleTV","ParentId":7027,"label":"Toronto Catholic District School Board / Secondary / StBasil_0513 / AppleTV"},{"LocationGroupId":7028,"LocationGroupName":"iOS","ParentId":7027,"label":"Toronto Catholic District School Board / Secondary / StBasil_0513 / iOS"},{"LocationGroupId":18997,"LocationGroupName":"iOSLease","ParentId":7027,"label":"Toronto Catholic District School Board / Secondary / StBasil_0513 / iOSLease"},{"LocationGroupId":6795,"LocationGroupName":"StJohnPaulII_0531","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / StJohnPaulII_0531"},{"LocationGroupId":18998,"LocationGroupName":"AppleTV","ParentId":6795,"label":"Toronto Catholic District School Board / Secondary / StJohnPaulII_0531 / AppleTV"},{"LocationGroupId":6796,"LocationGroupName":"iOS","ParentId":6795,"label":"Toronto Catholic District School Board / Secondary / StJohnPaulII_0531 / iOS"},{"LocationGroupId":18999,"LocationGroupName":"iOSLease","ParentId":6795,"label":"Toronto Catholic District School Board / Secondary / StJohnPaulII_0531 / iOSLease"},{"LocationGroupId":7102,"LocationGroupName":"StJosephCollege_0514","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / StJosephCollege_0514"},{"LocationGroupId":19000,"LocationGroupName":"AppleTV","ParentId":7102,"label":"Toronto Catholic District School Board / Secondary / StJosephCollege_0514 / AppleTV"},{"LocationGroupId":7133,"LocationGroupName":"iOS","ParentId":7102,"label":"Toronto Catholic District School Board / Secondary / StJosephCollege_0514 / iOS"},{"LocationGroupId":19002,"LocationGroupName":"iOSLease","ParentId":7102,"label":"Toronto Catholic District School Board / Secondary / StJosephCollege_0514 / iOSLease"},{"LocationGroupId":7029,"LocationGroupName":"StJosephMorrowPark_0516","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / StJosephMorrowPark_0516"},{"LocationGroupId":19003,"LocationGroupName":"AppleTV","ParentId":7029,"label":"Toronto Catholic District School Board / Secondary / StJosephMorrowPark_0516 / AppleTV"},{"LocationGroupId":7030,"LocationGroupName":"iOS","ParentId":7029,"label":"Toronto Catholic District School Board / Secondary / StJosephMorrowPark_0516 / iOS"},{"LocationGroupId":19004,"LocationGroupName":"iOSLease","ParentId":7029,"label":"Toronto Catholic District School Board / Secondary / StJosephMorrowPark_0516 / iOSLease"},{"LocationGroupId":7107,"LocationGroupName":"StMary_0528","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / StMary_0528"},{"LocationGroupId":19005,"LocationGroupName":"AppleTV","ParentId":7107,"label":"Toronto Catholic District School Board / Secondary / StMary_0528 / AppleTV"},{"LocationGroupId":7138,"LocationGroupName":"iOS","ParentId":7107,"label":"Toronto Catholic District School Board / Secondary / StMary_0528 / iOS"},{"LocationGroupId":19006,"LocationGroupName":"iOSLease","ParentId":7107,"label":"Toronto Catholic District School Board / Secondary / StMary_0528 / iOSLease"},{"LocationGroupId":4114,"LocationGroupName":"StMichaelChoirJr_Sr_0518","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / StMichaelChoirJr_Sr_0518"},{"LocationGroupId":19007,"LocationGroupName":"AppleTV","ParentId":4114,"label":"Toronto Catholic District School Board / Secondary / StMichaelChoirJr_Sr_0518 / AppleTV"},{"LocationGroupId":6739,"LocationGroupName":"iOS","ParentId":4114,"label":"Toronto Catholic District School Board / Secondary / StMichaelChoirJr_Sr_0518 / iOS"},{"LocationGroupId":19008,"LocationGroupName":"iOSLease","ParentId":4114,"label":"Toronto Catholic District School Board / Secondary / StMichaelChoirJr_Sr_0518 / iOSLease"},{"LocationGroupId":7119,"LocationGroupName":"StPatrick_0546","ParentId":2722,"label":"Toronto Catholic District School Board / Secondary / StPatrick_0546"},{"LocationGroupId":19009,"LocationGroupName":"AppleTV","ParentId":7119,"label":"Toronto Catholic District School Board / Secondary / StPatrick_0546 / AppleTV"},{"LocationGroupId":7144,"LocationGroupName":"iOS","ParentId":7119,"label":"Toronto Catholic District School Board / Secondary / StPatrick_0546 / iOS"},{"LocationGroupId":19010,"LocationGroupName":"iOSLease","ParentId":7119,"label":"Toronto Catholic District School Board / Secondary / StPatrick_0546 / iOSLease"},{"LocationGroupId":7077,"LocationGroupName":"Telecom","ParentId":2673,"label":"Toronto Catholic District School Board / Telecom"},{"LocationGroupId":16394,"LocationGroupName":"Telecom_iPad","ParentId":7077,"label":"Toronto Catholic District School Board / Telecom / Telecom_iPad"},{"LocationGroupId":16384,"LocationGroupName":"Telecom_iPhone","ParentId":7077,"label":"Toronto Catholic District School Board / Telecom / Telecom_iPhone"},{"LocationGroupId":18066,"LocationGroupName":"LostPhones","ParentId":16384,"label":"Toronto Catholic District School Board / Telecom / Telecom_iPhone / LostPhones"},{"LocationGroupId":7162,"LocationGroupName":"Telecom_R","ParentId":7077,"label":"Toronto Catholic District School Board / Telecom / Telecom_R"},{"LocationGroupId":16391,"LocationGroupName":"Wifi_Dedicated","ParentId":2673,"label":"Toronto Catholic District School Board / Wifi_Dedicated"},{"LocationGroupId":16393,"LocationGroupName":"Wifi_Android","ParentId":16391,"label":"Toronto Catholic District School Board / Wifi_Dedicated / Wifi_Android"},{"LocationGroupId":16392,"LocationGroupName":"Wifi_iPad","ParentId":16391,"label":"Toronto Catholic District School Board / Wifi_Dedicated / Wifi_iPad"}]'
    str_json=json.loads(grp)
    
    ioslease=None
    tv=None
    ios=None
    tcdsb=None 
 
    for s in str_json:
        if schoolcode in s['label']:
            #print s['label'].lower()[-3:]
            if 'ioslease' in s['label'].lower(): 
                ioslease= str(s['LocationGroupId']) 
            if 'ios' == s['label'].lower()[-3:]:
                ios=str(s['LocationGroupId'])
            if 'appletv' in s['label'].lower():
                tv=str(s['LocationGroupId']) 
        
                
    for g in lst:
        changeurl1='https://cn239.awmdm.com/AirWatch/Devices/ChangeDeviceLocationGroupMulti?selectedPrimaryKeys='+','.join(g.values())    
        #changeurl2='https://cn239.awmdm.com/AirWatch/Devices/ChangeDeviceLocationGroupMulti'
         
        browser.open(changeurl1)
        form=browser.get_form()
         
        if group_type=='ios':
            form['NewLocationGroupId']=ios
            print 'moving to '+str(ios)+' group'
            
        if group_type=='tv':
            form['NewLocationGroupId']=tv
            print 'moving to ios '+str(tv)+' group'
            
        if group_type=='ioslease':
            form['NewLocationGroupId']=ioslease
            print 'moving to '+str(ioslease)+' group'
            
        if group_type==OUs.tcdsb.name:
            print 'moving to '+OUs.tcdsb.name+' group'
            form['NewLocationGroupId']=OUs.tcdsb.value
            
        if group_type==OUs.elementary.name:
            print 'moving to '+OUs.elementary.name+' group'
            form['NewLocationGroupId']=OUs.elementary.value
            
        if group_type==OUs.secondary.name:
            print 'moving to '+OUs.secondary.name+' group'
            form['NewLocationGroupId']=OUs.secondary.value
          
        browser.submit_form(form)
        resp = json.load(StringIO(browser.response.content))

        #print resp
        _success=resp['IsSuccess']

        if _success == False:
            print "Changing group failed...."
            sys.exit(resp)
        else: 
            print "Changing group is successful on "+str(len(g))+" devices: "+str(g.keys())

def createBrowser(user,passwd):
    
    
    browser = RoboBrowser(history=True)
    
    browser.open('https://cn239.awmdm.com/AirWatch/Login/Login/Login-User')
    
    loginForm = browser.get_form()
    loginForm['UserName']=user
    loginForm['Password']=passwd
    
    browser.submit_form(loginForm)
    resp = json.load(StringIO(browser.response.content))
    
    _success=resp['IsSuccess']
    
    if _success == False:
        print "Login failed...."
        sys.exit(1)
    
    return browser


def setRole(role_id,browser):
    print 'setting OU to '+role_id
    RoleCookies={
    'ic4DUh/NXVp8VOKAtyDgbA==':'Elementary_Top_Level',
    'MVHG2D97GH8n5jFRanbzhQ==':'TCDSB_Top_Level',
    '0od0Xt3gw7UG3qUZKSaZvA==':'Secondary_Top_Level'
    }
    
    browser.open('https://cn239.awmdm.com/AirWatch/')
    token=findToken(browser)
    
    url='https://cn239.awmdm.com/AirWatch/Users/set-role'
    data={'id':role_id,'__RequestVerificationToken':token}
    browser.session.headers['X-Requested-With']='XMLHttpRequest'
    browser.session.headers['Accept']='*/*' 
    browser.session.headers['Host']='cn239.awmdm.com'
    browser.session.headers['Origin']='https://cn239.awmdm.com'
    browser.session.headers['Pragma']='no-cache'
    browser.session.headers['Referer']='https://cn239.awmdm.com/AirWatch/'
    browser.session.headers['Connection']='keep-alive' 
    browser.session.headers['Content-Type']='application/x-www-form-urlencoded; charset=UTF-8'
                            
    #cookies_old=browser.session.cookies.get_dict()
    oldRole=str(browser.session.cookies['BrandingUserLocationGroupID']).strip()
    
    if oldRole in RoleCookies.keys():
        print 'Old Role: '+ str(RoleCookies[oldRole])
     
    
    #browser.open('https://cn239.awmdm.com/AirWatch/Users/set-role?id='+role_id+'&__RequestVerificationToken='+token,method='post')
    browser.session.post('https://cn239.awmdm.com/AirWatch/Users/set-role',data)
    
    newRole=str(browser.session.cookies['BrandingUserLocationGroupID']).strip()
    
    if newRole in RoleCookies.keys():
        print 'Set new role to :'+str(RoleCookies[newRole])
        
def findToken(browser):
    
    data  = browser.find_all("script")
    paragraphs = []
    
    for x in data:
        paragraphs.append(str(x)) 
     
    regex='(?<=value=&quot;).{151}'
    matches = re.finditer(regex, " ".join(str(x) for x in paragraphs)) 
    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1

        #print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
        #print match.group()
        return match.group()
    
#         for groupNum in range(0, len(match.groups())):
#             groupNum = groupNum + 1

#             print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))


        
            

def main():
    
#     with open('sns.json', 'r') as f:
#         try:
#             devices = json.load(f)
#         # if the file is empty the ValueError will be thrown
#         except ValueError:
#             print 'Importing sns.json error'
#             devices = {}

    
    schoolcode=raw_input("school code:")
    sns=[]
    schoolcode=schoolcode.strip()
    
  
    # user = raw_input("Username:")
    # passwd = getpass.getpass("Password for " + user + ":")
    user='tcdsb.org\jiangk'
    passwd='2793587j'
    
    browser=createBrowser(user,passwd) 
    
    #setRole(Roles.Elementary.value,browser)
    
    count=0
    err_devices=[] 
    setRole(Roles.Elementary.value,browser)
    devices=searchAll(browser,schoolcode)
     
    print 'Found '+str(len(devices))+' matches'
    print '\n'
    print 'Moving devices out of OU' 
#     try: 
    moveOU(browser,devices,schoolcode,OUs.elementary.name)   
#     count+=1
#     except:
#             err_devices.append(k)
           
#     print 'Proccessed '+str(count)+' devices'
#     if len(err_devices)>0:
#         print 'Devices with errors'
#         for d in err_devices:
#             print d
    print 'Waiting for 1 minute...'     
    time.sleep(60)
    print 'Moving devices back into original OU'
#     try: 
    moveOU(browser,devices,schoolcode,OUs.ioslease.name)   
#     count+=1
#     except:
#             err_devices.append(k)
            
    
#     print 'Proccessed '+str(count)+' devices'
#     if len(err_devices)>0:
#         print 'Devices with errors'
#         for d in err_devices:
#             print d
                   
    print  "Finished....."
    

if __name__ == '__main__':
    main() 

school code:0344
setting OU to 18536
Old Role: Elementary_Top_Level
Set new role to :Elementary_Top_Level
Number of devices found: 21


SystemExit: Didn't find any devices matching naming convention

To exit: use 'exit', 'quit', or Ctrl-D.


In [103]:
pattern = re.compile("^[A,a]\d{4}[a-zA-Z0-9]{12}$") 
string ='A0276DMPTWCXYHLF9'
if pattern.match(string)!=None:
    print 'not none' 
else: 
    print 'none'

not none


In [16]:
s='A0278F7NMK6SFFP84 iPad iOS 9.3.5 FP84'
print s.split()[0]
schoolcode='0278'
pattern = re.compile("^[A,a]"+schoolcode.strip()+"[a-zA-Z0-9]{12}$") 
type(pattern.match(s.split()[0]))


A0278F7NMK6SFFP84


_sre.SRE_Match

In [61]:
len(deviceIds)

19